In [ ]:
import random
import matplotlib.pyplot as plt
import copy
# import networkx as nx

In [ ]:

class Graph:
    def __init__ (self, node_num):
        self.node_num = node_num
        # self.node_count = range(node_num) ## not needed

        ## need to change it to adjacency list for better read-time for edges ===> ALSO change 'add_edge' accordingly
        # self.adj_matrix =  [[0 for column in range(node_num)] for row in range(node_num)] ## adjacency matrix initialised

        self.adj_list = {node : set() for node in range(node_num)}
        self.visual = [] ## VIS

        i=0
        while i<node_num-1:
            self.add_edge(i, i+1)
            i=i+1
        self.add_edge(self.node_num-1,0)    
        
        self.add_rand_edges()

        # self.prey_belief = [1/49]*50
        # self.prey_belief[self.loc] = 0

    def add_edge(self, node1, node2): ## --> O/P: updates adj_list
        
        ## since graph is undirected, we add bi-directional edges

        # self.adj_matrix[node1][node2] = weight
        # self.adj_matrix[node2][node1] = weight
        
        self.adj_list[node1].add(node2)
        self.adj_list[node2].add(node1)

        tmp = [node1, node2] ## VIS
        self.visual.append(tmp) ## VIS
        
        ## if self.directed == False:
        ##    self.adj_matrix[node2][node1] = weight

    def get_deg(self, node): ## --> O/P: INT ## func to get degree of a node
        return len(self.adj_list[node])

    def get_next_moves(self, node):
        return list(self.adj_list[node]) 

    def get_valid_neighbours(self, node): ## --> O/P: list of neighbors (e.g.: [1, 2, 3, ...] )

        valid_neighbours = []
        # make a list of valid neighbors (-5 <- node -> 5)
        # make a new list from prev list with neighbors w/ degree = 2
        # random.choice from the new list
        ## I CAN INITIALISE i=-5 and then run till i<6 
        ##
        for i in range(-5, -1):
            next = (node + i + self.node_num) % self.node_num ## the "+ self.node_num" handles negative cases
            valid_neighbours.append(next)
        for i in range(2, 6):
            next = (node + i + self.node_num) % self.node_num ## the "+ self.node_num" handles negative cases
            valid_neighbours.append(next)

        return valid_neighbours
    
    def get_deg2_neighbours(self, node): ## --> O/P: list of neighbors with deg=2 [e.g.: [1, 2, 3, ...]]

        # make a new list from prev list with neighbors w/ degree = 2
        # random.choice from the new list
        valid_neighbours = self.get_valid_neighbours(node)
        deg2_neighbours = []
        for i in valid_neighbours:
            if(self.get_deg(i) == 2):
                deg2_neighbours.append(i)

        # print(valid_neighbours)
        # print("\n")
        # print(deg2_neighbours)
        return deg2_neighbours

    def get_rand_node(self):
        return random.choice(list(self.adj_list.keys()))

    def add_rand_edges(self): ## --> O/P: updates adj_list
        
        total_nodes = list(self.adj_list.keys()) ## --> total_nodes = [0,1,......,10]
        while(total_nodes):
            x = random.choice(total_nodes)
            # print (x , ":", total_nodes)
            valid_nbrs = self.get_deg2_neighbours(x)
            if valid_nbrs:
                y = random.choice(valid_nbrs) ## failing where it doesn't get deg2_neighbours
                self.add_edge(x, y)
                total_nodes.remove(y) ### fails where 'y' is NULL
                # print(y)
            total_nodes.remove(x)

    def getPath(self, goal, start, parent):

        result = [goal]
        while goal != start:
            goal = parent[goal]    
            result.append(goal)
        # print (goal)

        result.reverse()
        return result

    def BFS(self, start, goal):

        fringe = [start]
        visited = set()
        parent = {}
        visited.add(start)
        
        while(fringe):
            current = fringe.pop(0) #Queue so pop(0) implementing FIFO
            # visited.add(current) 
            if current == goal:
            #print(len(visited))
                return self.getPath(goal, start, parent)
            #print (current,"current")
            children = self.get_next_moves(current)
            #print(children,"test_BFS")
            for child in children:
                if child not in visited:
                    visited.add(child)
                    fringe.append(child)
                    parent[child] = current

        return None

    def print_adj_list(self):
        for i in self.adj_list.keys():
            print(i, self.adj_list[i])

    ## VIS
    def visualize(self):
        G = nx.Graph()
        G.add_edges_from(self.visual)
        nx.draw_networkx(G)
        plt.show()

In [ ]:
class Prey:
    def __init__(self, graph):
        self.loc = graph.get_rand_node()
        self.graph = graph

    # def get_neighbours(self):
    #     return self.graph.adj_list[self.loc] ## --> neighbours is a set()
        
    def move(self):
        # neighbours = list(self.get_neighbours())
        neighbours = self.graph.get_next_moves(self.loc)
        neighbours.append(self.loc)
        next = random.choice(neighbours)
        self.loc = next
        return self.loc

    def __str__(self):
        return str(self.loc)

    def __repr__(self):
        return self.loc

In [ ]:
class Predator:
    def __init__(self, graph):
        self.loc = graph.get_rand_node()
        self.graph = graph

    def move(self, agent_loc):
        # assert self.loc != agent_loc ## but will this prevent agent from dying ??

        neighbours = self.graph.get_next_moves(self.loc)

        path = self.graph.BFS(self.loc, agent_loc)
        self.loc = path[1]
        return self.loc

    def __str__(self):
        return str(self.loc)

    def __repr__(self):
        return self.loc
        

In [ ]:
class Agent1:

    def __init__(self, graph, prey_loc, pred_loc, dbg=False):
        
        self.graph = graph
        self.loc = graph.get_rand_node()
        self.dbg = dbg
        # self.loc = graph.get_rand_node()
        # print("Agent1: ", prey_loc, pred_loc)
        while prey_loc == self.loc or pred_loc == self.loc:
            self.loc = graph.get_rand_node()

    def debug(self, *args):
        if self.dbg:
            print("Agent1: ", args)

    def isPrey(self, prey_loc):
        if self.loc == prey_loc:
            return True

    def isPred(self, pred_loc):
        if self.loc == pred_loc:
            return True

    def move(self, prey_loc, pred_loc):
        
        self.loc = self.move_helper(prey_loc, pred_loc)
        return self.loc

    def move_helper(self, prey_loc, pred_loc):
        nbrs = self.graph.get_next_moves(self.loc)
        prey_path_dict = {}
        pred_path_dict = {}

        curr_path_prey = len(self.graph.BFS(self.loc, prey_loc))
        curr_path_pred = len(self.graph.BFS(self.loc, pred_loc))
        self.debug("Curr Path:", curr_path_prey, curr_path_pred)

        for node in nbrs:
            prey_path_dict[node] = len(self.graph.BFS(node, prey_loc)) ## {(1:3), (2:4)}
            pred_path_dict[node] = len(self.graph.BFS(node, pred_loc))

        self.debug("Prey Paths:", prey_path_dict)
        self.debug("Pred Paths:", pred_path_dict)

        nodes_lt_prey = []
        nodes_eq_prey = []
        for n, cost in prey_path_dict.items():
            if cost < curr_path_prey:
                nodes_lt_prey.append(n)
            elif cost == curr_path_prey:
                nodes_eq_prey.append(n)

        nodes_gt_pred = []
        nodes_eq_pred = []
        for n, cost in pred_path_dict.items():
            if cost > curr_path_pred:
                nodes_gt_pred.append(n)
            elif cost == curr_path_pred:
                nodes_eq_pred.append(n)

        valid_n = []

        ## 1. closer to prey and farther from pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 1")
                return random.choice(valid_n)
        
        ## 2. closer to prey and not closer to pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 2")
                return random.choice(valid_n)

        ## 3. not farther than prey and farther from pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 3")
                return random.choice(valid_n)

        ## 4. not farther than prey and not closer to pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 4")
                return random.choice(valid_n)
        
        ## 5. farther from pred
        if nodes_gt_pred:
            self.debug("Rule 5")
            return random.choice(nodes_gt_pred)

        ## 6. not closer to pred
        if nodes_eq_pred:
            self.debug("Rule 6")
            return random.choice(nodes_eq_pred)

        ## 7. sit still and pray
        else:
            self.debug("Rule 7")
            return self.loc


In [ ]:
class Agent2:

    def __init__(self, graph, prey_loc, pred_loc, dbg=False):

        self.graph = graph
        self.loc = graph.get_rand_node()
        self.dbg = dbg
        # self.loc = graph.get_rand_node()
        # print("Agent1: ", prey_loc, pred_loc)
        while prey_loc == self.loc or pred_loc == self.loc:
            self.loc = graph.get_rand_node()

    def debug(self, *args):
        if self.dbg:
            print("Agent2: ", args)

    def isPrey(self, prey_loc):
        if self.loc == prey_loc:
            return True

    def isPred(self, pred_loc):
        if self.loc == pred_loc:
            return True
    
    def move(self, prey_loc, pred_loc):
        self.loc = self.move_helper(prey_loc, pred_loc)
        return self.loc

    def move_helper(self,prey_loc, pred_loc):
        
        nbrs = self.graph.get_next_moves(self.loc)
        prey_path_dict = {}
        pred_path_dict = {}

        curr_path_prey = len(self.graph.BFS(self.loc, prey_loc))
        curr_path_pred = len(self.graph.BFS(self.loc, pred_loc))
        self.debug("Curr Path:", curr_path_prey, curr_path_pred)
        for node in nbrs:
            prey_path_dict[node] = len(self.graph.BFS(node, prey_loc)) ## {(1:3), (2:4)}
            pred_path_dict[node] = len(self.graph.BFS(node, pred_loc))

        self.debug("Prey Paths:", prey_path_dict)
        self.debug("Pred Paths:", pred_path_dict)
        nodes_lt_prey = []
        nodes_eq_prey = []
        for n, cost in prey_path_dict.items():
            if cost < curr_path_prey:
                nodes_lt_prey.append(n)
            elif cost == curr_path_prey:
                nodes_eq_prey.append(n)

        nodes_gt_pred = []
        nodes_eq_pred = []
        for n, cost in pred_path_dict.items():
            if cost > curr_path_pred:
                nodes_gt_pred.append(n)
            elif cost == curr_path_pred:
                nodes_eq_pred.append(n)

        valid_n = []

        ## 3. not farther than prey and farther from pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 3")
                return random.choice(valid_n)
        
        ## 5. farther from pred
        if nodes_gt_pred:
            self.debug("Rule 5")
            return random.choice(nodes_gt_pred)

        ## 1. closer to prey and farther from pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 1")
                return random.choice(valid_n)
        
        ## 2. closer to prey and not closer to pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 2")
                return random.choice(valid_n)
        
        ## 4. not farther than prey and not closer to pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 4")
                return random.choice(valid_n)

        ## 6. not closer to pred
        if nodes_eq_pred:
            self.debug("Rule 6")
            return random.choice(nodes_eq_pred)

        ## 7. sit still and pray
        else:
            self.debug("Rule 7")
            return self.loc


In [ ]:
class Agent3:

    def __init__(self, graph, prey_loc, pred_loc, prey_belief, dbg=False):
        
        self.graph = graph
        self.loc = graph.get_rand_node()
        self.dbg = dbg
        # self.loc = graph.get_rand_node()
        # print("Agent1: ", prey_loc, pred_loc)
        self.prey_loc = prey_loc
        self.pred_loc = pred_loc

        while prey_loc == self.loc or pred_loc == self.loc:
            self.loc = graph.get_rand_node()

        # self.prey_belief = [1/49]*50
        self.prey_belief = prey_belief
        self.prey_belief[self.loc] = 0

        # self.new_prey_belief = []
    
    # def update_prey_belief(self, prey_belief):

    def debug(self, *args):
        if self.dbg:
            print("Agent3: ", args)

    def isPrey(self, loc, prey_loc):
        if loc == prey_loc:
            return True

    def isPred(self, loc, pred_loc):
        if loc == pred_loc:
            return True

    def survey(self, prey_belief):
        ## look at current list of belief - select max prob nodes (if multiple - select one at random)
        new_prey_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        val_max = max(prey_belief)
        # idx_max = 0 ## not needed
        idx_max_list = []

        for i in range(len(prey_belief)):
            if prey_belief[i] >= val_max:
                # val_max = prey_belief[i]
                idx_max_list.append(i)

        rand_idx_max = random.choice(idx_max_list)
        # print(prey_belief, val_max, idx_max_list)

        # ### check if prey is there or not
        # 
        if self.isPrey(rand_idx_max, self.prey_loc):
            # print("IS PREY EXECUTED: ", rand_idx_max)
            for i in range(self.graph.node_num):
                prey_belief[i] = 0.0
            prey_belief[rand_idx_max] = 1.0
            # print ("ISPREY: ", prey_belief)
            return prey_belief
        #     update_belief() ## according to prey found in survey logic
                
        else:
        #     update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                # denom = sum(prey_belief)-prey_belief[rand_idx_max]
                denom = 1-prey_belief[rand_idx_max]
                new_prey_belief[i] = prey_belief[i]/denom
                new_prey_belief[rand_idx_max] = 0.0

        prey_belief = new_prey_belief
        return prey_belief

    def agent_move_prey_belief(self, prey_loc, prey_belief):

        new_prey_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        if self.isPrey(self.loc, prey_loc):
            for i in range(self.graph.node_num): ###///this will not actually happen since the game would be over
                prey_belief[i] = 0.0
            prey_belief[self.loc] = 1.0
            return prey_belief  
        
        else:
        #   update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                # denom = sum(prey_belief)-prey_belief[self.loc]
                denom = 1.0-prey_belief[self.loc]
                new_prey_belief[i] = prey_belief[i]/denom
                new_prey_belief[self.loc] = 0.0
                # new_prey_belief[curr_loc] = 0.0
                # prey_belief[i] = prey_belief[i]/denom

        prey_belief = new_prey_belief
        return prey_belief

    def prey_move_prey_belief(self, prey_loc, prey_belief):

        new_prey_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        for i in range(self.graph.node_num):
            nbrs_i = self.graph.get_next_moves(i)
            nbrs_i.append(i)
            for nbr in nbrs_i:
                denom = self.graph.get_deg(nbr) + 1
                new_prey_belief[i] += prey_belief[nbr] / denom

        prey_belief = new_prey_belief

        prey_belief = self.agent_move_prey_belief(prey_loc, prey_belief)

        # prey_belief = new_prey_belief
        return prey_belief

    # ## #### #### ####     
    # def prey_move_prey_belief(self, prey_loc, prey_belief):
    #     new_prey_belief = []
    #     for node in range(self.graph.node_num):
    #         tmp = 0.0
    #         for nbr in self.graph.get_next_moves(node):
    #             tmp += prey_belief[nbr] / (self.graph.get_deg(nbr)+1)
    #         tmp += prey_belief[node]/(self.graph.get_deg(node)+1)
    #         new_prey_belief.append(tmp)
    #     return new_prey_belief

    def move(self, prey_loc, pred_loc, prey_belief):
        self.debug('Prey Belief: ', prey_belief, " ; SUM :", sum(prey_belief))
        prey_belief = self.survey(prey_belief)
        self.debug('Prey Belief after S: ', prey_belief, " ; SUM :", sum(prey_belief))
        curr_loc = self.loc ######
        self.loc = self.move_helper(prey_loc, pred_loc, prey_belief)
        prey_belief = self.agent_move_prey_belief(prey_loc, prey_belief)
        # prey_belief[curr_loc] = 0.0
        self.debug('Prey Belief after A: ', prey_belief, " ; SUM :", sum(prey_belief))
        # prey_belief = self.prey_move_prey_belief(prey3.loc, prey_belief)
        # self.debug('Prey Belief after P: ', prey_belief, " ; SUM :", sum(prey_belief))
        return self.loc

    # def move_helper(self, prey_loc, pred_loc, prey_belief):
    def move_helper(self, prey_loc, pred_loc, prey_belief):

        ## We select max belief node from list of max belief nodes and assume it to be where Prey is even if its not been found
        val_max = max(prey_belief)
        idx_max = 0
        idx_max_list = []

        for i in range(len(prey_belief)):
            if prey_belief[i] >= val_max:
                val_max = prey_belief[i]
                idx_max_list.append(i)

        prey_loc = random.choice(idx_max_list) ##need self.prey_loc or not? // why is prey_loc being or not being there as a parameter affecting code ??????
        self.debug('Predicted prey location :', prey_loc)
        
        nbrs = self.graph.get_next_moves(self.loc)
        prey_path_dict = {}
        pred_path_dict = {}

        curr_path_prey = len(self.graph.BFS(self.loc, prey_loc))
        curr_path_pred = len(self.graph.BFS(self.loc, pred_loc))
        self.debug("Curr Path:", curr_path_prey, curr_path_pred)

        for node in nbrs:
            prey_path_dict[node] = len(self.graph.BFS(node, prey_loc)) ## {(1:3), (2:4)}
            pred_path_dict[node] = len(self.graph.BFS(node, pred_loc))

        self.debug("Prey Paths:", prey_path_dict)
        self.debug("Pred Paths:", pred_path_dict)

        nodes_lt_prey = []
        nodes_eq_prey = []
        for n, cost in prey_path_dict.items():
            if cost < curr_path_prey:
                nodes_lt_prey.append(n)
            elif cost == curr_path_prey:
                nodes_eq_prey.append(n)

        nodes_gt_pred = []
        nodes_eq_pred = []
        for n, cost in pred_path_dict.items():
            if cost > curr_path_pred:
                nodes_gt_pred.append(n)
            elif cost == curr_path_pred:
                nodes_eq_pred.append(n)

        valid_n = []

        ## 1. closer to prey and farther from pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 1")
                return random.choice(valid_n)
        
        ## 2. closer to prey and not closer to pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 2")
                return random.choice(valid_n)

        ## 3. not farther than prey and farther from pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 3")
                return random.choice(valid_n)

        ## 4. not farther than prey and not closer to pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 4")
                return random.choice(valid_n)
        
        ## 5. farther from pred
        if nodes_gt_pred:
            self.debug("Rule 5")
            return random.choice(nodes_gt_pred)

        ## 6. not closer to pred
        if nodes_eq_pred:
            self.debug("Rule 6")
            return random.choice(nodes_eq_pred)

        ## 7. sit still and pray
        else:
            self.debug("Rule 7")
            return self.loc
        

In [ ]:
class Agent4:

    def __init__(self, graph, prey_loc, pred_loc, prey_belief, dbg=False):
        
        self.graph = graph
        self.loc = graph.get_rand_node()
        self.dbg = dbg
        self.prey_loc = prey_loc
        self.pred_loc = pred_loc

        while prey_loc == self.loc or pred_loc == self.loc:
            self.loc = graph.get_rand_node()

        self.prey_belief = prey_belief
        self.prey_belief[self.loc] = 0

    
    def debug(self, *args):
        if self.dbg:
            print("Agent4: ", args)


    def isPrey(self, loc, prey_loc):
        if loc == prey_loc:
            return True


    def isPred(self, loc, pred_loc):
        if loc == pred_loc:
            return True


    def survey(self, prey_belief):
        ## look at current list of belief - select max prob nodes (if multiple - select one at random)
        new_prey_belief = [0.0]*(self.graph.node_num)

        val_max = max(prey_belief)
        idx_max_list = []

        for i in range(len(prey_belief)):
            if prey_belief[i] >= val_max:
                idx_max_list.append(i)

        rand_idx_max = random.choice(idx_max_list)

        # ### check if prey is there or not
        # 
        if self.isPrey(rand_idx_max, self.prey_loc):
            for i in range(self.graph.node_num):
                prey_belief[i] = 0.0
            prey_belief[rand_idx_max] = 1.0
            return prey_belief
        #     update_belief() ## according to prey found in survey logic
                
        else:
        #     update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                denom = 1-prey_belief[rand_idx_max]
                new_prey_belief[i] = prey_belief[i]/denom
                new_prey_belief[rand_idx_max] = 0.0

        prey_belief = new_prey_belief
        return prey_belief


    def agent_move_prey_belief(self, prey_loc, prey_belief):

        new_prey_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        if self.isPrey(self.loc, prey_loc):
            for i in range(self.graph.node_num): ###///this will not actually happen since the game would be over
                prey_belief[i] = 0.0
            prey_belief[self.loc] = 1.0
            return prey_belief  
        
        else:
        #   update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                denom = 1.0-prey_belief[self.loc]
                new_prey_belief[i] = prey_belief[i]/denom
                new_prey_belief[self.loc] = 0.0

        prey_belief = new_prey_belief
        return prey_belief


    def prey_move_prey_belief(self, prey_loc, prey_belief):

        new_prey_belief = [0.0]*(self.graph.node_num)

        for i in range(self.graph.node_num):
            nbrs_i = self.graph.get_next_moves(i)
            nbrs_i.append(i)
            for nbr in nbrs_i:
                denom = self.graph.get_deg(nbr) + 1
                new_prey_belief[i] += prey_belief[nbr] / denom

        prey_belief = new_prey_belief

        prey_belief = self.agent_move_prey_belief(prey_loc, prey_belief)

        return prey_belief


    def move(self, prey_loc, pred_loc, prey_belief, tmp_prey_belief):
        self.debug('Prey Belief: ', prey_belief, " ; SUM :", sum(prey_belief))
        prey_belief = self.survey(prey_belief)
        self.debug('Prey Belief after S: ', prey_belief, " ; SUM :", sum(prey_belief))
        
        self.loc = self.move_helper(prey_loc, pred_loc, tmp_prey_belief)
        
        prey_belief = self.agent_move_prey_belief(prey_loc, prey_belief)
        # prey_belief[curr_loc] = 0.0
        self.debug('Prey Belief after A: ', prey_belief, " ; SUM :", sum(prey_belief))
        # prey_belief = self.prey_move_prey_belief(prey3.loc, prey_belief)
        # self.debug('Prey Belief after P: ', prey_belief, " ; SUM :", sum(prey_belief))
        return self.loc


    def move_helper(self, prey_loc, pred_loc, prey_belief):

        ## We select max belief node from list of max belief nodes and assume it to be where Prey is even if its not been found
        val_max = max(prey_belief)
        idx_max = 0
        idx_max_list = []

        for i in range(len(prey_belief)):
            if prey_belief[i] >= val_max:
                val_max = prey_belief[i]
                idx_max_list.append(i)

        prey_loc = random.choice(idx_max_list) ##need self.prey_loc or not? // why is prey_loc being or not being there as a parameter affecting code ??????
        self.debug('Predicted prey location :', prey_loc)
        
        nbrs = self.graph.get_next_moves(self.loc)
        prey_path_dict = {}
        pred_path_dict = {}

        curr_path_prey = len(self.graph.BFS(self.loc, prey_loc))
        curr_path_pred = len(self.graph.BFS(self.loc, pred_loc))
        self.debug("Curr Path:", curr_path_prey, curr_path_pred)

        for node in nbrs:
            prey_path_dict[node] = len(self.graph.BFS(node, prey_loc)) ## {(1:3), (2:4)}
            pred_path_dict[node] = len(self.graph.BFS(node, pred_loc))

        self.debug("Prey Paths:", prey_path_dict)
        self.debug("Pred Paths:", pred_path_dict)

        nodes_lt_prey = []
        nodes_eq_prey = []
        for n, cost in prey_path_dict.items():
            if cost < curr_path_prey:
                nodes_lt_prey.append(n)
            elif cost == curr_path_prey:
                nodes_eq_prey.append(n)

        nodes_gt_pred = []
        nodes_eq_pred = []
        for n, cost in pred_path_dict.items():
            if cost > curr_path_pred:
                nodes_gt_pred.append(n)
            elif cost == curr_path_pred:
                nodes_eq_pred.append(n)

        valid_n = []

        ## 1. closer to prey and farther from pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 1")
                return random.choice(valid_n)
        
        ## 2. closer to prey and not closer to pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 2")
                return random.choice(valid_n)

        ## 3. not farther than prey and farther from pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 3")
                return random.choice(valid_n)

        ## 4. not farther than prey and not closer to pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 4")
                return random.choice(valid_n)
        
        ## 5. farther from pred
        if nodes_gt_pred:
            self.debug("Rule 5")
            return random.choice(nodes_gt_pred)

        ## 6. not closer to pred
        if nodes_eq_pred:
            self.debug("Rule 6")
            return random.choice(nodes_eq_pred)

        ## 7. sit still and pray
        else:
            self.debug("Rule 7")
            return self.loc
        

In [ ]:
class Agent5:

    def __init__(self, graph, prey_loc, pred_loc, pred_belief, dbg=False):
        
        self.graph = graph
        self.loc = graph.get_rand_node()
        self.dbg = dbg
        # self.loc = graph.get_rand_node()
        # print("Agent1: ", prey_loc, pred_loc)
        self.prey_loc = prey_loc
        self.pred_loc = pred_loc

        while prey_loc == self.loc or pred_loc == self.loc:
            self.loc = graph.get_rand_node()

        # self.pred_belief = [1/49]*50
        self.pred_belief = pred_belief
        self.pred_belief[self.loc] = 0

    def debug(self, *args):
        if self.dbg:
            print("Agent5: ", args)

    def isPrey(self, loc, prey_loc):
        if loc == prey_loc:
            return True

    def isPred(self, loc, pred_loc):
        if loc == pred_loc:
            return True

    def survey(self, pred_belief):
        ## look at current list of belief - select max prob nodes (if multiple - select one at random)
        new_pred_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        val_max = max(pred_belief)
        # idx_max = 0 ## not needed
        idx_max_list = []

        for i in range(len(pred_belief)):
            if pred_belief[i] >= val_max:
                # val_max = prey_belief[i]
                idx_max_list.append(i)

        # tmp_max_list = []
        dist_list = []
        dist_min_list = []

        for node in idx_max_list:
            dist_list.append(len(self.graph.BFS(node, self.pred_loc)))
        
        min_dist = min(dist_list) ## CALCULATING MINIMUM dist from agent to partcular pred_probable posi

        for i in range(len(dist_list)):
            if dist_list[i] <= min_dist:
                dist_min_list.append(i)

        rand_idx_max = random.choice(dist_min_list)
        # print(prey_belief, val_max, idx_max_list)

        # ### check if prey is there or not
        # 
        if self.isPred(rand_idx_max, self.pred_loc):
            # print("IS PREY EXECUTED: ", rand_idx_max)
            for i in range(self.graph.node_num):
                pred_belief[i] = 0.0
            pred_belief[rand_idx_max] = 1.0
            # print ("ISPREY: ", prey_belief)
            return pred_belief
        #     update_belief() ## according to prey found in survey logic
                
        else:
        #     update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                # denom = sum(prey_belief)-prey_belief[rand_idx_max]
                denom = 1-pred_belief[rand_idx_max]
                new_pred_belief[i] = pred_belief[i]/denom
                new_pred_belief[rand_idx_max] = 0.0

        pred_belief = new_pred_belief
        return pred_belief

    def agent_move_pred_belief(self, pred_loc, pred_belief):

        new_pred_belief = [0.0]*(self.graph.node_num)

        if self.isPred(self.loc, pred_loc):
            for i in range(self.graph.node_num): ###///this will not actually happen since the game would be over
                pred_belief[i] = 0.0
            pred_belief[self.loc] = 1.0
            return pred_belief  
        
        else:
        #   update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                # denom = sum(prey_belief)-prey_belief[self.loc]
                denom = 1.0-pred_belief[self.loc]
                new_pred_belief[i] = pred_belief[i]/denom
                new_pred_belief[self.loc] = 0.0
                # new_prey_belief[curr_loc] = 0.0
                # prey_belief[i] = prey_belief[i]/denom

        pred_belief = new_pred_belief
        return pred_belief

    def pred_move_pred_belief(self, pred_loc, pred_belief):
        pred_belief_tmp = [0.0]*self.graph.node_num
        # print ("before func PRED_MOVE_UPDATE : ", sum(pred_belief))
        for i in range(self.graph.node_num):
            tmp_belief = 0.0

            nbrs_i = self.graph.get_next_moves(i)
            
            for nbr in nbrs_i:
                nbr_bfs = {}
                
                nbr_nbrs_i = self.graph.get_next_moves(nbr)
                for nbr_nbr in nbr_nbrs_i:
                    bfs = self.graph.BFS(nbr_nbr, self.loc)
                    nbr_bfs[nbr_nbr] = len(bfs)

                # print ("NBR_BFS: ", nbr_bfs)
                min_bfs = min(nbr_bfs.values())

                nbr_min_path_len = [key for key, value in nbr_bfs.items() if value == min_bfs]
                # print("nbr_min_path_len : ", nbr_min_path_len)

                x = len(nbr_min_path_len)
                # print ("DEBUG: ", i, " ", nbr_min_path_len)
                if i in nbr_min_path_len:
                    tmp_belief += pred_belief[nbr]*((0.4/self.graph.get_deg(nbr))+(0.6/x))
                    # print("tmp bel in 1 ", tmp_belief)
                else:
                    tmp_belief += pred_belief[nbr]*((0.4/self.graph.get_deg(nbr)))
                    # print("tmp bel in 2 ", tmp_belief)

                # print("i :", i)
                # print("nbrs_i", nbrs_i)
                # print("nbr_nbr", nbr_nbrs_i)

                # exit (0)
            # print("TMP BELIEF: ", tmp_belief)
            pred_belief_tmp[i] = tmp_belief

        # print ("after func PRED_MOVE_UPDATE OLD : ", pred_belief)

        # print ("after func PRED_MOVE_UPDATE : ", sum(pred_belief_tmp))

        return pred_belief_tmp


    def move(self, prey_loc, pred_loc, pred_belief):
        self.debug('Pred Belief: ', pred_belief, " ; SUM :", sum(pred_belief))
        pred_belief = self.survey(pred_belief)
        self.debug('Pred Belief after S: ', pred_belief, " ; SUM :", sum(pred_belief))
        curr_loc = self.loc ######
        self.loc = self.move_helper(prey_loc, pred_loc, pred_belief)
        pred_belief = self.agent_move_pred_belief(pred_loc, pred_belief)
        # prey_belief[curr_loc] = 0.0
        self.debug('Pred Belief after A: ', pred_belief, " ; SUM :", sum(pred_belief))

        return self.loc


    def move_helper(self, prey_loc, pred_loc, pred_belief):

        #####///// WE SELECT MAX BELIEF NODE for pred_loc
        val_max = max(pred_belief)
        idx_max = 0
        idx_max_list = []

        for i in range(len(pred_belief)):
            if pred_belief[i] >= val_max:
                # val_max = prey_belief[i]
                idx_max_list.append(i)

        pred_loc = random.choice(idx_max_list) ##need self.prey_loc or not? // why is prey_loc being or not being there as a parameter affecting code ??????
        self.debug('Predicted pred location :', pred_loc)
        
        nbrs = self.graph.get_next_moves(self.loc)
        prey_path_dict = {}
        pred_path_dict = {}

        curr_path_prey = len(self.graph.BFS(self.loc, prey_loc))
        curr_path_pred = len(self.graph.BFS(self.loc, pred_loc))
        self.debug("Curr Path:", curr_path_prey, curr_path_pred)

        for node in nbrs:
            prey_path_dict[node] = len(self.graph.BFS(node, prey_loc)) ## {(1:3), (2:4)}
            pred_path_dict[node] = len(self.graph.BFS(node, pred_loc))

        self.debug("Prey Paths:", prey_path_dict)
        self.debug("Pred Paths:", pred_path_dict)

        nodes_lt_prey = []
        nodes_eq_prey = []
        for n, cost in prey_path_dict.items():
            if cost < curr_path_prey:
                nodes_lt_prey.append(n)
            elif cost == curr_path_prey:
                nodes_eq_prey.append(n)

        nodes_gt_pred = []
        nodes_eq_pred = []
        for n, cost in pred_path_dict.items():
            if cost > curr_path_pred:
                nodes_gt_pred.append(n)
            elif cost == curr_path_pred:
                nodes_eq_pred.append(n)

        valid_n = []

        ## 1. closer to prey and farther from pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 1")
                return random.choice(valid_n)
        
        ## 2. closer to prey and not closer to pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 2")
                return random.choice(valid_n)

        ## 3. not farther than prey and farther from pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 3")
                return random.choice(valid_n)

        ## 4. not farther than prey and not closer to pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 4")
                return random.choice(valid_n)
        
        ## 5. farther from pred
        if nodes_gt_pred:
            self.debug("Rule 5")
            return random.choice(nodes_gt_pred)

        ## 6. not closer to pred
        if nodes_eq_pred:
            self.debug("Rule 6")
            return random.choice(nodes_eq_pred)

        ## 7. sit still and pray
        else:
            self.debug("Rule 7")
            return self.loc
        

In [ ]:
class Agent6:

    def __init__(self, graph, prey_loc, pred_loc, pred_belief, dbg=False):
        
        self.graph = graph
        self.loc = graph.get_rand_node()
        self.dbg = dbg
        # self.loc = graph.get_rand_node()
        # print("Agent1: ", prey_loc, pred_loc)
        self.prey_loc = prey_loc
        self.pred_loc = pred_loc

        while prey_loc == self.loc or pred_loc == self.loc:
            self.loc = graph.get_rand_node()

        # self.pred_belief = [1/49]*50
        self.pred_belief = pred_belief
        self.pred_belief[self.loc] = 0

    def debug(self, *args):
        if self.dbg:
            print("Agent6: ", args)

    def isPrey(self, loc, prey_loc):
        if loc == prey_loc:
            return True

    def isPred(self, loc, pred_loc):
        if loc == pred_loc:
            return True

    def survey(self, pred_belief):
        ## look at current list of belief - select max prob nodes (if multiple - select one at random)
        new_pred_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        val_max = max(pred_belief)
        # idx_max = 0 ## not needed
        idx_max_list = []

        for i in range(len(pred_belief)):
            if pred_belief[i] >= val_max:
                # val_max = prey_belief[i]
                idx_max_list.append(i)

        # tmp_max_list = []
        dist_list = []
        dist_min_list = []

        for node in idx_max_list:
            dist_list.append(len(self.graph.BFS(node, self.pred_loc)))
        
        min_dist = min(dist_list) ## CALCULATING MINIMUM dist from agent to partcular pred_probable posi

        for i in range(len(dist_list)):
            if dist_list[i] <= min_dist:
                dist_min_list.append(i)

        rand_idx_max = random.choice(dist_min_list)
        # print(prey_belief, val_max, idx_max_list)

        # ### check if prey is there or not
        # 
        if self.isPred(rand_idx_max, self.pred_loc):
            # print("IS PREY EXECUTED: ", rand_idx_max)
            for i in range(self.graph.node_num):
                pred_belief[i] = 0.0
            pred_belief[rand_idx_max] = 1.0
            # print ("ISPREY: ", prey_belief)
            return pred_belief
        #     update_belief() ## according to prey found in survey logic
                
        else:
        #     update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                # denom = sum(prey_belief)-prey_belief[rand_idx_max]
                denom = 1-pred_belief[rand_idx_max]
                new_pred_belief[i] = pred_belief[i]/denom
                new_pred_belief[rand_idx_max] = 0.0

        pred_belief = new_pred_belief
        return pred_belief

    def agent_move_pred_belief(self, pred_loc, pred_belief):

        new_pred_belief = [0.0]*(self.graph.node_num)

        if self.isPred(self.loc, pred_loc):
            for i in range(self.graph.node_num): ###///this will not actually happen since the game would be over
                pred_belief[i] = 0.0
            pred_belief[self.loc] = 1.0
            return pred_belief  
        
        else:
        #   update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                # denom = sum(prey_belief)-prey_belief[self.loc]
                denom = 1.0-pred_belief[self.loc]
                new_pred_belief[i] = pred_belief[i]/denom
                new_pred_belief[self.loc] = 0.0
                # new_prey_belief[curr_loc] = 0.0
                # prey_belief[i] = prey_belief[i]/denom

        pred_belief = new_pred_belief
        return pred_belief

    def pred_move_pred_belief(self, pred_loc, pred_belief):
        pred_belief_tmp = [0.0]*self.graph.node_num
        # print ("before func PRED_MOVE_UPDATE : ", sum(pred_belief))
        for i in range(self.graph.node_num):
            tmp_belief = 0.0

            nbrs_i = self.graph.get_next_moves(i)
            
            for nbr in nbrs_i:
                nbr_bfs = {}
                
                nbr_nbrs_i = self.graph.get_next_moves(nbr)
                for nbr_nbr in nbr_nbrs_i:
                    bfs = self.graph.BFS(nbr_nbr, self.loc)
                    nbr_bfs[nbr_nbr] = len(bfs)

                # print ("NBR_BFS: ", nbr_bfs)
                min_bfs = min(nbr_bfs.values())

                nbr_min_path_len = [key for key, value in nbr_bfs.items() if value == min_bfs]
                # print("nbr_min_path_len : ", nbr_min_path_len)

                x = len(nbr_min_path_len)
                # print ("DEBUG: ", i, " ", nbr_min_path_len)
                if i in nbr_min_path_len:
                    tmp_belief += pred_belief[nbr]*((0.4/self.graph.get_deg(nbr))+(0.6/x))
                    # print("tmp bel in 1 ", tmp_belief)
                else:
                    tmp_belief += pred_belief[nbr]*((0.4/self.graph.get_deg(nbr)))
                    # print("tmp bel in 2 ", tmp_belief)

                # print("i :", i)
                # print("nbrs_i", nbrs_i)
                # print("nbr_nbr", nbr_nbrs_i)

                # exit (0)
            # print("TMP BELIEF: ", tmp_belief)
            pred_belief_tmp[i] = tmp_belief

        # print ("after func PRED_MOVE_UPDATE OLD : ", pred_belief)

        # print ("after func PRED_MOVE_UPDATE : ", sum(pred_belief_tmp))

        return pred_belief_tmp


    def move(self, prey_loc, pred_loc, pred_belief):
        self.debug('Pred Belief: ', pred_belief, " ; SUM :", sum(pred_belief))
        pred_belief = self.survey(pred_belief)
        self.debug('Pred Belief after S: ', pred_belief, " ; SUM :", sum(pred_belief))
        curr_loc = self.loc ######
        self.loc = self.move_helper(prey_loc, pred_loc, pred_belief)
        pred_belief = self.agent_move_pred_belief(pred_loc, pred_belief)
        # prey_belief[curr_loc] = 0.0
        self.debug('Pred Belief after A: ', pred_belief, " ; SUM :", sum(pred_belief))

        return self.loc


    def move_helper(self, prey_loc, pred_loc, pred_belief):

        #####///// WE SELECT MAX BELIEF NODE for pred_loc
        val_max = max(pred_belief)
        idx_max = 0
        idx_max_list = []

        for i in range(len(pred_belief)):
            if pred_belief[i] >= val_max:
                # val_max = prey_belief[i]
                idx_max_list.append(i)
                
        pred_loc = random.choice(idx_max_list) ##need self.prey_loc or not? // why is prey_loc being or not being there as a parameter affecting code ??????
        self.debug('Predicted pred location :', pred_loc)
        
        nbrs = self.graph.get_next_moves(self.loc)
        prey_path_dict = {}
        pred_path_dict = {}

        curr_path_prey = len(self.graph.BFS(self.loc, prey_loc))
        curr_path_pred = len(self.graph.BFS(self.loc, pred_loc))
        self.debug("Curr Path:", curr_path_prey, curr_path_pred)

        for node in nbrs:
            prey_path_dict[node] = len(self.graph.BFS(node, prey_loc)) ## {(1:3), (2:4)}
            pred_path_dict[node] = len(self.graph.BFS(node, pred_loc))

        self.debug("Prey Paths:", prey_path_dict)
        self.debug("Pred Paths:", pred_path_dict)

        nodes_lt_prey = []
        nodes_eq_prey = []
        for n, cost in prey_path_dict.items():
            if cost < curr_path_prey:
                nodes_lt_prey.append(n)
            elif cost == curr_path_prey:
                nodes_eq_prey.append(n)

        nodes_gt_pred = []
        nodes_eq_pred = []
        for n, cost in pred_path_dict.items():
            if cost > curr_path_pred:
                nodes_gt_pred.append(n)
            elif cost == curr_path_pred:
                nodes_eq_pred.append(n)

        valid_n = []

        # for i in range(len(idx_max_list)):
            # dist = abs(abs(idx_max_list[i] - self.loc)-self.graph.node_num) % self.graph.node_num
        dist = abs(abs(pred_loc - self.loc)-self.graph.node_num) % self.graph.node_num

        if dist >= 5:

            ## 1. closer to prey and farther from pred
            if nodes_lt_prey:
                for n in nodes_lt_prey:
                    if n in nodes_gt_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 1")
                    return random.choice(valid_n)
            
            ## 2. closer to prey and not closer to pred
            if nodes_lt_prey:
                for n in nodes_lt_prey:
                    if n in nodes_eq_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 2")
                    return random.choice(valid_n)

            ## 3. not farther than prey and farther from pred
            if nodes_eq_prey:
                for n in nodes_eq_prey:
                    if n in nodes_gt_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 3")
                    return random.choice(valid_n)

            ## 4. not farther than prey and not closer to pred
            if nodes_eq_prey:
                for n in nodes_eq_prey:
                    if n in nodes_eq_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 4")
                    return random.choice(valid_n)
            
            ## 5. farther from pred
            if nodes_gt_pred:
                self.debug("Rule 5")
                return random.choice(nodes_gt_pred)

            ## 6. not closer to pred
            if nodes_eq_pred:
                self.debug("Rule 6")
                return random.choice(nodes_eq_pred)

            ## 7. sit still and pray
            else:
                self.debug("Rule 7")
                return self.loc
        
        else:

            ## 3. not farther than prey and farther from pred
            if nodes_eq_prey:
                for n in nodes_eq_prey:
                    if n in nodes_gt_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 3")
                    return random.choice(valid_n)
            
            ## 5. farther from pred
            if nodes_gt_pred:
                self.debug("Rule 5")
                return random.choice(nodes_gt_pred)

            ## 1. closer to prey and farther from pred
            if nodes_lt_prey:
                for n in nodes_lt_prey:
                    if n in nodes_gt_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 1")
                    return random.choice(valid_n)
            
            ## 2. closer to prey and not closer to pred
            if nodes_lt_prey:
                for n in nodes_lt_prey:
                    if n in nodes_eq_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 2")
                    return random.choice(valid_n)
            
            ## 4. not farther than prey and not closer to pred
            if nodes_eq_prey:
                for n in nodes_eq_prey:
                    if n in nodes_eq_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 4")
                    return random.choice(valid_n)

            ## 6. not closer to pred
            if nodes_eq_pred:
                self.debug("Rule 6")
                return random.choice(nodes_eq_pred)

            ## 7. sit still and pray
            self.debug("Rule 7")
            return self.loc

    

In [ ]:
class Agent7:

    def __init__(self, graph, prey_loc, pred_loc, prey_belief, pred_belief, dbg=False):
        
        self.graph = graph
        self.loc = graph.get_rand_node()
        self.dbg = dbg
        # self.loc = graph.get_rand_node()
        # print("Agent1: ", prey_loc, pred_loc)
        self.prey_loc = prey_loc
        self.pred_loc = pred_loc

        while prey_loc == self.loc or pred_loc == self.loc:
            self.loc = graph.get_rand_node()

        # self.pred_belief = [1/49]*50
        self.pred_belief = pred_belief
        self.pred_belief[self.loc] = 0

        self.prey_belief = prey_belief
        self.prey_belief[self.loc] = 0

    def debug(self, *args):
        if self.dbg:
            print("Agent7: ", args)

    def isPrey(self, loc, prey_loc):
        if loc == prey_loc:
            return True

    def isPred(self, loc, pred_loc):
        if loc == pred_loc:
            return True

    def survey(self, pred_belief, prey_belief):

        # for i in range(len(self.graph.node_num)):
        #     if pred_belief[i] == 1.0

        if sum(pred_belief) in pred_belief: ##
            #### PREY CODE
            new_prey_belief = [0.0]*(self.graph.node_num)
            new_pred_belief = [0.0]*(self.graph.node_num)

            val_max = max(prey_belief)
            idx_max_list = []

            for i in range(len(prey_belief)):
                if prey_belief[i] >= val_max:
                    # val_max = prey_belief[i]
                    idx_max_list.append(i)

            rand_idx_max = random.choice(idx_max_list)

            # if self.isPred(rand_idx_max, self.pred.loc):
            #     FLAG = 1

            if self.isPrey(rand_idx_max, self.prey_loc):
                # print("IS PREY EXECUTED: ", rand_idx_max)
                for i in range(self.graph.node_num):
                    # prey_belief[i] = 0.0
                    new_prey_belief[i] = 0.0
                # prey_belief[rand_idx_max] = 1.0
                new_prey_belief[rand_idx_max] = 1.0
                # print ("ISPREY: ", prey_belief)
                # return prey_belief, pred_belief
                    
            else:
                for i in range(self.graph.node_num):
                    denom = 1-prey_belief[rand_idx_max]
                    new_prey_belief[i] = prey_belief[i]/denom
                new_prey_belief[rand_idx_max] = 0.0 ###changed indent(earlier in for loop)

            prey_belief = new_prey_belief

            # #################################

            if self.isPred(rand_idx_max, self.pred_loc):
                for i in range(self.graph.node_num):
                    # pred_belief[i] = 0.0
                    new_pred_belief[i] = 0.0
                # pred_belief[rand_idx_max] = 1.0
                new_pred_belief[rand_idx_max] = 1.0
                # return prey_belief, pred_belief

            else:
                for i in range(self.graph.node_num):
                    denom = 1-pred_belief[rand_idx_max]
                    new_pred_belief[i] = pred_belief[i]/denom
                new_pred_belief[rand_idx_max] = 0.0 ###changed indent(earlier in for loop)

            pred_belief = new_pred_belief

            # #################################

            return prey_belief, pred_belief     

        else:
            ### PRED CODE
            new_pred_belief = [0.0]*(self.graph.node_num)
            new_prey_belief = [0.0]*(self.graph.node_num)

            val_max = max(pred_belief)
            idx_max_list = []

            for i in range(len(pred_belief)):
                if pred_belief[i] >= val_max:
                    idx_max_list.append(i)

            dist_list = []
            dist_min_list = []

            for node in idx_max_list:
                dist_list.append(len(self.graph.BFS(node, self.pred_loc)))
            
            min_dist = min(dist_list) ## CALCULATING MINIMUM dist from agent to partcular pred_probable pos

            for i in range(len(dist_list)):
                if dist_list[i] <= min_dist:
                    dist_min_list.append(i)

            rand_idx_max = random.choice(dist_min_list)

            if self.isPred(rand_idx_max, self.pred_loc):
                for i in range(self.graph.node_num):
                    # pred_belief[i] = 0.0
                    new_pred_belief[i] = 0.0
                # pred_belief[rand_idx_max] = 1.0
                new_pred_belief[rand_idx_max] = 1.0
                # return prey_belief, pred_belief

            else:
                for i in range(self.graph.node_num):
                    denom = 1-pred_belief[rand_idx_max]
                    new_pred_belief[i] = pred_belief[i]/denom
                new_pred_belief[rand_idx_max] = 0.0 ###changed indent(earlier in for loop)

            pred_belief = new_pred_belief
            # return prey_belief, pred_belief

                        
            # ###########################

            if self.isPrey(rand_idx_max, self.prey_loc):
                # print("IS PREY EXECUTED: ", rand_idx_max)
                for i in range(self.graph.node_num):
                    # prey_belief[i] = 0.0
                    new_prey_belief[i] = 0.0
                # prey_belief[rand_idx_max] = 1.0
                new_prey_belief[rand_idx_max] = 1.0
                
                # print ("ISPREY: ", prey_belief)
                # return prey_belief, pred_belief

            else:
                for i in range(self.graph.node_num):
                    denom = 1-prey_belief[rand_idx_max]
                    new_prey_belief[i] = prey_belief[i]/denom
                new_prey_belief[rand_idx_max] = 0.0 ###changed indent(earlier in for loop)

            prey_belief = new_prey_belief
            # return prey_belief, pred_belief    

            # ###########################

            return prey_belief, pred_belief 

    

    def agent_move_pred_belief(self, pred_loc, pred_belief):

        new_pred_belief = [0.0]*(self.graph.node_num)

        if self.isPred(self.loc, pred_loc):
            for i in range(self.graph.node_num): ###///this will not actually happen since the game would be over
                # pred_belief[i] = 0.0
                new_pred_belief[i] = 0.0
            # pred_belief[self.loc] = 1.0
            new_pred_belief[self.loc] = 1.0
            # return pred_belief
        
        else:
            for i in range(self.graph.node_num):
                denom = 1.0-pred_belief[self.loc]
                new_pred_belief[i] = pred_belief[i]/denom
            new_pred_belief[self.loc] = 0.0 ###changed indent(earlier in for loop)
                # prey_belief[i] = prey_belief[i]/denom

        pred_belief = new_pred_belief
        return pred_belief

    def pred_move_pred_belief(self, pred_loc, pred_belief):
        pred_belief_tmp = [0.0]*self.graph.node_num
        # print ("before func PRED_MOVE_UPDATE : ", sum(pred_belief))
        for i in range(self.graph.node_num):
            tmp_belief = 0.0

            nbrs_i = self.graph.get_next_moves(i)
            
            for nbr in nbrs_i:
                nbr_bfs = {}
                
                nbr_nbrs_i = self.graph.get_next_moves(nbr)
                for nbr_nbr in nbr_nbrs_i:
                    bfs = self.graph.BFS(nbr_nbr, self.loc)
                    nbr_bfs[nbr_nbr] = len(bfs)

                # print ("NBR_BFS: ", nbr_bfs)
                min_bfs = min(nbr_bfs.values())

                nbr_min_path_len = [key for key, value in nbr_bfs.items() if value == min_bfs]
                # print("nbr_min_path_len : ", nbr_min_path_len)

                x = len(nbr_min_path_len)
                # print ("DEBUG: ", i, " ", nbr_min_path_len)
                if i in nbr_min_path_len:
                    tmp_belief += pred_belief[nbr]*((0.4/self.graph.get_deg(nbr))+(0.6/x))
                    # print("tmp bel in 1 ", tmp_belief)
                else:
                    tmp_belief += pred_belief[nbr]*((0.4/self.graph.get_deg(nbr)))
                    # print("tmp bel in 2 ", tmp_belief)

                # print ("i :", i)
                # print("nbrs_i", nbrs_i)
                # print("nbr_nbr", nbr_nbrs_i)

                # exit (0)
            # print("TMP BELIEF: ", tmp_belief)
            pred_belief_tmp[i] = tmp_belief

        # print ("after func PRED_MOVE_UPDATE OLD : ", pred_belief)

        # print ("after func PRED_MOVE_UPDATE : ", sum(pred_belief_tmp))

        return pred_belief_tmp


    def agent_move_prey_belief(self, prey_loc, prey_belief):

        new_prey_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        if self.isPrey(self.loc, prey_loc):
            for i in range(self.graph.node_num): ###///this will not actually happen since the game would be over
                # prey_belief[i] = 0.0
                new_prey_belief[i] = 0.0
            # prey_belief[self.loc] = 1.0
            new_prey_belief[self.loc] = 1.0
            # return prey_belief  
        
        else:
        #   update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                # denom = sum(prey_belief)-prey_belief[self.loc]
                denom = 1.0-prey_belief[self.loc]
                new_prey_belief[i] = prey_belief[i]/denom
            new_prey_belief[self.loc] = 0.0 ###changed indent(earlier in for loop)
                # new_prey_belief[curr_loc] = 0.0
                # prey_belief[i] = prey_belief[i]/denom

        prey_belief = new_prey_belief
        return prey_belief

    def prey_move_prey_belief(self, prey_loc, prey_belief):

        new_prey_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        for i in range(self.graph.node_num):
            nbrs_i = self.graph.get_next_moves(i)
            nbrs_i.append(i)
            for nbr in nbrs_i:
                denom = self.graph.get_deg(nbr) + 1
                new_prey_belief[i] += prey_belief[nbr] / denom

        prey_belief = new_prey_belief

        prey_belief = self.agent_move_prey_belief(prey_loc, prey_belief)

        # prey_belief = new_prey_belief
        return prey_belief


    def move(self, prey_loc, pred_loc, pred_belief, prey_belief):
        self.debug('Pred Belief: ', pred_belief, " ; SUM :", sum(pred_belief))
        self.debug('Prey Belief: ', prey_belief, " ; SUM :", sum(prey_belief))

        prey_belief, pred_belief = self.survey(pred_belief, prey_belief)
        self.debug('Pred Belief after S: ', pred_belief, " ; SUM :", sum(pred_belief))
        self.debug('Prey Belief after S: ', prey_belief, " ; SUM :", sum(prey_belief))
        curr_loc = self.loc ######
        self.loc = self.move_helper(prey_loc, pred_loc, prey_belief, pred_belief)
        prey_belief = self.agent_move_prey_belief(prey_loc, prey_belief)
        pred_belief = self.agent_move_pred_belief(pred_loc, pred_belief)

        # prey_belief[curr_loc] = 0.0
        self.debug('Pred Belief after A: ', pred_belief, " ; SUM :", sum(pred_belief))
        self.debug('Prey Belief after A: ', prey_belief, " ; SUM :", sum(prey_belief))

        return self.loc


    def move_helper(self, prey_loc, pred_loc, prey_belief, pred_belief):

        #####///// WE SELECT MAX BELIEF NODE for pred_loc
        val_max1 = max(pred_belief)
        idx_max1 = 0
        idx_max_list1 = []

        for i in range(len(pred_belief)):
            if pred_belief[i] >= val_max1:
                # val_max = prey_belief[i]
                idx_max_list1.append(i)

        pred_loc = random.choice(idx_max_list1) ##need self.prey_loc or not? // why is prey_loc being or not being there as a parameter affecting code ??????
        self.debug('Predicted pred location :', pred_loc)

        ## We select max belief node from list of max belief nodes and assume it to be where Prey is even if its not been found
        val_max = max(prey_belief)
        idx_max = 0
        idx_max_list = []

        for i in range(len(prey_belief)):
            if prey_belief[i] >= val_max:
                val_max = prey_belief[i]
                idx_max_list.append(i)

        prey_loc = random.choice(idx_max_list) ##need self.prey_loc or not? // why is prey_loc being or not being there as a parameter affecting code ??????
        self.debug('Predicted prey location :', prey_loc)
        
        nbrs = self.graph.get_next_moves(self.loc)
        prey_path_dict = {}
        pred_path_dict = {}

        curr_path_prey = len(self.graph.BFS(self.loc, prey_loc))
        curr_path_pred = len(self.graph.BFS(self.loc, pred_loc))
        self.debug("Curr Path:", curr_path_prey, curr_path_pred)

        for node in nbrs:
            prey_path_dict[node] = len(self.graph.BFS(node, prey_loc)) ## {(1:3), (2:4)}
            pred_path_dict[node] = len(self.graph.BFS(node, pred_loc))

        self.debug("Prey Paths:", prey_path_dict)
        self.debug("Pred Paths:", pred_path_dict)

        nodes_lt_prey = []
        nodes_eq_prey = []
        for n, cost in prey_path_dict.items():
            if cost < curr_path_prey:
                nodes_lt_prey.append(n)
            elif cost == curr_path_prey:
                nodes_eq_prey.append(n)

        nodes_gt_pred = []
        nodes_eq_pred = []
        for n, cost in pred_path_dict.items():
            if cost > curr_path_pred:
                nodes_gt_pred.append(n)
            elif cost == curr_path_pred:
                nodes_eq_pred.append(n)

        valid_n = []

        ## 1. closer to prey and farther from pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 1")
                return random.choice(valid_n)
        
        ## 2. closer to prey and not closer to pred
        if nodes_lt_prey:
            for n in nodes_lt_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 2")
                return random.choice(valid_n)

        ## 3. not farther than prey and farther from pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_gt_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 3")
                return random.choice(valid_n)

        ## 4. not farther than prey and not closer to pred
        if nodes_eq_prey:
            for n in nodes_eq_prey:
                if n in nodes_eq_pred:
                    valid_n.append(n)
            if valid_n:
                self.debug("Rule 4")
                return random.choice(valid_n)
        
        ## 5. farther from pred
        if nodes_gt_pred:
            self.debug("Rule 5")
            return random.choice(nodes_gt_pred)

        ## 6. not closer to pred
        if nodes_eq_pred:
            self.debug("Rule 6")
            return random.choice(nodes_eq_pred)

        ## 7. sit still and pray
        else:
            self.debug("Rule 7")
            return self.loc
        

In [ ]:
class Agent8:

    def __init__(self, graph, prey_loc, pred_loc, prey_belief, pred_belief, dbg=False):
        
        self.graph = graph
        self.loc = graph.get_rand_node()
        self.dbg = dbg
        # self.loc = graph.get_rand_node()
        # print("Agent1: ", prey_loc, pred_loc)
        self.prey_loc = prey_loc
        self.pred_loc = pred_loc

        while prey_loc == self.loc or pred_loc == self.loc:
            self.loc = graph.get_rand_node()

        # self.pred_belief = [1/49]*50
        self.pred_belief = pred_belief
        self.pred_belief[self.loc] = 0

        self.prey_belief = prey_belief
        self.prey_belief[self.loc] = 0

    def debug(self, *args):
        if self.dbg:
            print("Agent7: ", args)

    def isPrey(self, loc, prey_loc):
        if loc == prey_loc:
            return True

    def isPred(self, loc, pred_loc):
        if loc == pred_loc:
            return True

    def survey(self, pred_belief, prey_belief):

        # for i in range(len(self.graph.node_num)):
        #     if pred_belief[i] == 1.0

        if sum(pred_belief) in pred_belief: ##
            #### PREY CODE
            new_prey_belief = [0.0]*(self.graph.node_num)
            new_pred_belief = [0.0]*(self.graph.node_num)

            val_max = max(prey_belief)
            idx_max_list = []

            for i in range(len(prey_belief)):
                if prey_belief[i] >= val_max:
                    # val_max = prey_belief[i]
                    idx_max_list.append(i)

            rand_idx_max = random.choice(idx_max_list)

            # if self.isPred(rand_idx_max, self.pred.loc):
            #     FLAG = 1

            if self.isPrey(rand_idx_max, self.prey_loc):
                # print("IS PREY EXECUTED: ", rand_idx_max)
                for i in range(self.graph.node_num):
                    # prey_belief[i] = 0.0
                    new_prey_belief[i] = 0.0
                # prey_belief[rand_idx_max] = 1.0
                new_prey_belief[rand_idx_max] = 1.0
                # print ("ISPREY: ", prey_belief)
                # return prey_belief, pred_belief
                    
            else:
                for i in range(self.graph.node_num):
                    denom = 1-prey_belief[rand_idx_max]
                    new_prey_belief[i] = prey_belief[i]/denom
                new_prey_belief[rand_idx_max] = 0.0 ###changed indent(earlier in for loop)

            prey_belief = new_prey_belief

            # #################################

            if self.isPred(rand_idx_max, self.pred_loc):
                for i in range(self.graph.node_num):
                    # pred_belief[i] = 0.0
                    new_pred_belief[i] = 0.0
                # pred_belief[rand_idx_max] = 1.0
                new_pred_belief[rand_idx_max] = 1.0
                # return prey_belief, pred_belief

            else:
                for i in range(self.graph.node_num):
                    denom = 1-pred_belief[rand_idx_max]
                    new_pred_belief[i] = pred_belief[i]/denom
                new_pred_belief[rand_idx_max] = 0.0 ###changed indent(earlier in for loop)

            pred_belief = new_pred_belief

            # #################################

            return prey_belief, pred_belief     

        else:
            ### PRED CODE
            new_pred_belief = [0.0]*(self.graph.node_num)
            new_prey_belief = [0.0]*(self.graph.node_num)

            val_max = max(pred_belief)
            idx_max_list = []

            for i in range(len(pred_belief)):
                if pred_belief[i] >= val_max:
                    idx_max_list.append(i)

            dist_list = []
            dist_min_list = []

            for node in idx_max_list:
                dist_list.append(len(self.graph.BFS(node, self.pred_loc)))
            
            min_dist = min(dist_list) ## CALCULATING MINIMUM dist from agent to partcular pred_probable pos

            for i in range(len(dist_list)):
                if dist_list[i] <= min_dist:
                    dist_min_list.append(i)

            rand_idx_max = random.choice(dist_min_list)

            if self.isPred(rand_idx_max, self.pred_loc):
                for i in range(self.graph.node_num):
                    # pred_belief[i] = 0.0
                    new_pred_belief[i] = 0.0
                # pred_belief[rand_idx_max] = 1.0
                new_pred_belief[rand_idx_max] = 1.0
                # return prey_belief, pred_belief

            else:
                for i in range(self.graph.node_num):
                    denom = 1-pred_belief[rand_idx_max]
                    new_pred_belief[i] = pred_belief[i]/denom
                new_pred_belief[rand_idx_max] = 0.0 ###changed indent(earlier in for loop)

            pred_belief = new_pred_belief
            # return prey_belief, pred_belief

                        
            # ###########################

            if self.isPrey(rand_idx_max, self.prey_loc):
                # print("IS PREY EXECUTED: ", rand_idx_max)
                for i in range(self.graph.node_num):
                    # prey_belief[i] = 0.0
                    new_prey_belief[i] = 0.0
                # prey_belief[rand_idx_max] = 1.0
                new_prey_belief[rand_idx_max] = 1.0
                
                # print ("ISPREY: ", prey_belief)
                # return prey_belief, pred_belief

            else:
                for i in range(self.graph.node_num):
                    denom = 1-prey_belief[rand_idx_max]
                    new_prey_belief[i] = prey_belief[i]/denom
                new_prey_belief[rand_idx_max] = 0.0 ###changed indent(earlier in for loop)

            prey_belief = new_prey_belief
            # return prey_belief, pred_belief    

            # ###########################

            return prey_belief, pred_belief 

    

    def agent_move_pred_belief(self, pred_loc, pred_belief):

        new_pred_belief = [0.0]*(self.graph.node_num)

        if self.isPred(self.loc, pred_loc):
            for i in range(self.graph.node_num): ###///this will not actually happen since the game would be over
                # pred_belief[i] = 0.0
                new_pred_belief[i] = 0.0
            # pred_belief[self.loc] = 1.0
            new_pred_belief[self.loc] = 1.0
            # return pred_belief
        
        else:
            for i in range(self.graph.node_num):
                denom = 1.0-pred_belief[self.loc]
                new_pred_belief[i] = pred_belief[i]/denom
            new_pred_belief[self.loc] = 0.0 ###changed indent(earlier in for loop)
                # prey_belief[i] = prey_belief[i]/denom

        pred_belief = new_pred_belief
        return pred_belief

    def pred_move_pred_belief(self, pred_loc, pred_belief):
        pred_belief_tmp = [0.0]*self.graph.node_num
        # print ("before func PRED_MOVE_UPDATE : ", sum(pred_belief))
        for i in range(self.graph.node_num):
            tmp_belief = 0.0

            nbrs_i = self.graph.get_next_moves(i)
            
            for nbr in nbrs_i:
                nbr_bfs = {}
                
                nbr_nbrs_i = self.graph.get_next_moves(nbr)
                for nbr_nbr in nbr_nbrs_i:
                    bfs = self.graph.BFS(nbr_nbr, self.loc)
                    nbr_bfs[nbr_nbr] = len(bfs)

                # print ("NBR_BFS: ", nbr_bfs)
                min_bfs = min(nbr_bfs.values())

                nbr_min_path_len = [key for key, value in nbr_bfs.items() if value == min_bfs]
                # print("nbr_min_path_len : ", nbr_min_path_len)

                x = len(nbr_min_path_len)
                # print ("DEBUG: ", i, " ", nbr_min_path_len)
                if i in nbr_min_path_len:
                    tmp_belief += pred_belief[nbr]*((0.4/self.graph.get_deg(nbr))+(0.6/x))
                    # print("tmp bel in 1 ", tmp_belief)
                else:
                    tmp_belief += pred_belief[nbr]*((0.4/self.graph.get_deg(nbr)))
                    # print("tmp bel in 2 ", tmp_belief)

                # print ("i :", i)
                # print("nbrs_i", nbrs_i)
                # print("nbr_nbr", nbr_nbrs_i)

                # exit (0)
            # print("TMP BELIEF: ", tmp_belief)
            pred_belief_tmp[i] = tmp_belief

        # print ("after func PRED_MOVE_UPDATE OLD : ", pred_belief)

        # print ("after func PRED_MOVE_UPDATE : ", sum(pred_belief_tmp))

        return pred_belief_tmp


    def agent_move_prey_belief(self, prey_loc, prey_belief):

        new_prey_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        if self.isPrey(self.loc, prey_loc):
            for i in range(self.graph.node_num): ###///this will not actually happen since the game would be over
                # prey_belief[i] = 0.0
                new_prey_belief[i] = 0.0
            # prey_belief[self.loc] = 1.0
            new_prey_belief[self.loc] = 1.0
            # return prey_belief  
        
        else:
        #   update_belief() ## according to prey not found in survey logic
            for i in range(self.graph.node_num):
                # denom = sum(prey_belief)-prey_belief[self.loc]
                denom = 1.0-prey_belief[self.loc]
                new_prey_belief[i] = prey_belief[i]/denom
            new_prey_belief[self.loc] = 0.0 ###changed indent(earlier in for loop)
                # new_prey_belief[curr_loc] = 0.0
                # prey_belief[i] = prey_belief[i]/denom

        prey_belief = new_prey_belief
        return prey_belief

    def prey_move_prey_belief(self, prey_loc, prey_belief):

        new_prey_belief = [0.0]*(self.graph.node_num)
        # new_prey_belief = prey_belief

        for i in range(self.graph.node_num):
            nbrs_i = self.graph.get_next_moves(i)
            nbrs_i.append(i)
            for nbr in nbrs_i:
                denom = self.graph.get_deg(nbr) + 1
                new_prey_belief[i] += prey_belief[nbr] / denom

        prey_belief = new_prey_belief

        prey_belief = self.agent_move_prey_belief(prey_loc, prey_belief)

        # prey_belief = new_prey_belief
        return prey_belief


    def move(self, prey_loc, pred_loc, pred_belief, prey_belief):
        self.debug('Pred Belief: ', pred_belief, " ; SUM :", sum(pred_belief))
        self.debug('Prey Belief: ', prey_belief, " ; SUM :", sum(prey_belief))

        prey_belief, pred_belief = self.survey(pred_belief, prey_belief)
        self.debug('Pred Belief after S: ', pred_belief, " ; SUM :", sum(pred_belief))
        self.debug('Prey Belief after S: ', prey_belief, " ; SUM :", sum(prey_belief))
        curr_loc = self.loc ######
        self.loc = self.move_helper(prey_loc, pred_loc, prey_belief, pred_belief)
        prey_belief = self.agent_move_prey_belief(prey_loc, prey_belief)
        pred_belief = self.agent_move_pred_belief(pred_loc, pred_belief)

        # prey_belief[curr_loc] = 0.0
        self.debug('Pred Belief after A: ', pred_belief, " ; SUM :", sum(pred_belief))
        self.debug('Prey Belief after A: ', prey_belief, " ; SUM :", sum(prey_belief))

        return self.loc


    def move_helper(self, prey_loc, pred_loc, prey_belief, pred_belief):

        #####///// WE SELECT MAX BELIEF NODE for pred_loc
        val_max1 = max(pred_belief)
        idx_max1 = 0
        idx_max_list1 = []

        for i in range(len(pred_belief)):
            if pred_belief[i] >= val_max1:
                # val_max = prey_belief[i]
                idx_max_list1.append(i)

        pred_loc = random.choice(idx_max_list1) ##need self.prey_loc or not? // why is prey_loc being or not being there as a parameter affecting code ??????
        self.debug('Predicted pred location :', pred_loc)

        ## We select max belief node from list of max belief nodes and assume it to be where Prey is even if its not been found
        val_max = max(prey_belief)
        idx_max = 0
        idx_max_list = []

        for i in range(len(prey_belief)):
            if prey_belief[i] >= val_max:
                val_max = prey_belief[i]
                idx_max_list.append(i)

        prey_loc = random.choice(idx_max_list) ##need self.prey_loc or not? // why is prey_loc being or not being there as a parameter affecting code ??????
        self.debug('Predicted prey location :', prey_loc)
        
        nbrs = self.graph.get_next_moves(self.loc)
        prey_path_dict = {}
        pred_path_dict = {}

        curr_path_prey = len(self.graph.BFS(self.loc, prey_loc))
        curr_path_pred = len(self.graph.BFS(self.loc, pred_loc))
        self.debug("Curr Path:", curr_path_prey, curr_path_pred)

        for node in nbrs:
            prey_path_dict[node] = len(self.graph.BFS(node, prey_loc)) ## {(1:3), (2:4)}
            pred_path_dict[node] = len(self.graph.BFS(node, pred_loc))

        self.debug("Prey Paths:", prey_path_dict)
        self.debug("Pred Paths:", pred_path_dict)

        nodes_lt_prey = []
        nodes_eq_prey = []
        for n, cost in prey_path_dict.items():
            if cost < curr_path_prey:
                nodes_lt_prey.append(n)
            elif cost == curr_path_prey:
                nodes_eq_prey.append(n)

        nodes_gt_pred = []
        nodes_eq_pred = []
        for n, cost in pred_path_dict.items():
            if cost > curr_path_pred:
                nodes_gt_pred.append(n)
            elif cost == curr_path_pred:
                nodes_eq_pred.append(n)

        valid_n = []
        
        dist = abs(abs(pred_loc - self.loc)-self.graph.node_num) % self.graph.node_num

        if dist >= 5:

            ## 1. closer to prey and farther from pred
            if nodes_lt_prey:
                for n in nodes_lt_prey:
                    if n in nodes_gt_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 1")
                    return random.choice(valid_n)
            
            ## 2. closer to prey and not closer to pred
            if nodes_lt_prey:
                for n in nodes_lt_prey:
                    if n in nodes_eq_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 2")
                    return random.choice(valid_n)

            ## 3. not farther than prey and farther from pred
            if nodes_eq_prey:
                for n in nodes_eq_prey:
                    if n in nodes_gt_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 3")
                    return random.choice(valid_n)

            ## 4. not farther than prey and not closer to pred
            if nodes_eq_prey:
                for n in nodes_eq_prey:
                    if n in nodes_eq_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 4")
                    return random.choice(valid_n)
            
            ## 5. farther from pred
            if nodes_gt_pred:
                self.debug("Rule 5")
                return random.choice(nodes_gt_pred)

            ## 6. not closer to pred
            if nodes_eq_pred:
                self.debug("Rule 6")
                return random.choice(nodes_eq_pred)

            ## 7. sit still and pray
            else:
                self.debug("Rule 7")
                return self.loc
        
        else:

            ## 3. not farther than prey and farther from pred
            if nodes_eq_prey:
                for n in nodes_eq_prey:
                    if n in nodes_gt_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 3")
                    return random.choice(valid_n)
            
            ## 5. farther from pred
            if nodes_gt_pred:
                self.debug("Rule 5")
                return random.choice(nodes_gt_pred)

            ## 1. closer to prey and farther from pred
            if nodes_lt_prey:
                for n in nodes_lt_prey:
                    if n in nodes_gt_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 1")
                    return random.choice(valid_n)
            
            ## 2. closer to prey and not closer to pred
            if nodes_lt_prey:
                for n in nodes_lt_prey:
                    if n in nodes_eq_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 2")
                    return random.choice(valid_n)
            
            ## 4. not farther than prey and not closer to pred
            if nodes_eq_prey:
                for n in nodes_eq_prey:
                    if n in nodes_eq_pred:
                        valid_n.append(n)
                if valid_n:
                    self.debug("Rule 4")
                    return random.choice(valid_n)

            ## 6. not closer to pred
            if nodes_eq_pred:
                self.debug("Rule 6")
                return random.choice(nodes_eq_pred)

            ## 7. sit still and pray
            self.debug("Rule 7")
            return self.loc


In [ ]:
node_num = 50
g = Graph(node_num)
# g.print_adj_list()

# g.visualize()

prey = Prey(g)
# prey2 = prey1
# prey2 = copy.deepcopy(prey1)
prey1 = copy.copy(prey)
prey2 = copy.copy(prey) ## ???? should I do deep or shallow copy of prey and pred if I want them to be separate for each agent but not have to write new code ?????
prey3 = copy.copy(prey)
prey4 = copy.copy(prey)
prey5 = copy.copy(prey)
prey6 = copy.copy(prey)
prey7 = copy.copy(prey)
prey8 = copy.copy(prey)
prey9 = copy.copy(prey)

pred = Predator(g)
# pred2 = pred1
# pred2 = copy.deepcopy(pred1)
pred1 = copy.copy(pred)
pred2 = copy.copy(pred)
pred3 = copy.copy(pred)
pred4 = copy.copy(pred)
pred5 = copy.copy(pred)
pred6 = copy.copy(pred)
pred7 = copy.copy(pred)
pred8 = copy.copy(pred)
pred9 = copy.copy(pred)

prey_belief = [1/(g.node_num-1)]*(g.node_num) ### [1/49]*50
pred_belief = [1/(g.node_num-1)]*(g.node_num)

threshold = 1000
loops = 1000 ### CHANGE TO 3000

In [ ]:
# threshold = 100
node_num = 50

cnt = 0
hung = 0
ans = 0

survival = []

for i in range(loops):
    g = Graph(node_num)
    prey = Prey(g)
    pred = Predator(g)
    prey8 = copy.copy(prey)
    pred8 = copy.copy(pred)

    
    prey_belief = [1/(g.node_num-1)]*(g.node_num) ### [1/49]*50
    pred_belief = [1/(g.node_num-1)]*(g.node_num)

    ##------ AGENT-8 RUN MAIN ------##
    agent8 = Agent8(g, prey8.loc, pred8.loc, prey_belief, pred_belief, False)

    count = 0

    while agent8.loc != pred8.loc and agent8.loc != prey8.loc:
        if count > threshold:
            print("HUNG SIMULATION")
            # break
            hung += 1
            continue

        agent8.debug("BEFORE: ", agent8.loc, prey8.loc, pred8.loc)
        agent8.loc = agent8.move(prey8.loc, pred8.loc, pred_belief, prey_belief)

        if agent8.isPred(agent8.loc, pred8.loc) and agent8.isPrey(agent8.loc, prey8.loc):
            print(True)
            ans += 1
            continue
        elif agent8.isPred(agent8.loc, pred8.loc):
            print(False)
            continue
        elif agent8.isPrey(agent8.loc, prey8.loc):
            print(True)
            ans += 1
            continue

        prey8.loc = prey8.move()
        prey_belief = agent8.prey_move_prey_belief(prey8.loc, prey_belief)
        pred8.loc = pred8.move(agent8.loc)
        pred_belief = agent8.pred_move_pred_belief(pred8.loc, pred_belief)
        agent8.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
        
        agent8.debug("After: ", agent8.loc, prey8.loc, pred8.loc) ##
        agent8.debug("prey8 found : ", agent8.isPrey(agent8.loc, prey8.loc)," ; ", "pred5 found : ", agent8.isPred(agent8.loc, pred8.loc)) ##

        count = count + 1

        if agent8.isPred(agent8.loc, pred8.loc) and agent8.isPrey(agent8.loc, prey8.loc):
            print(True)
            ans += 1
            continue
        elif agent8.isPred(agent8.loc, pred8.loc):
            print(False)
            continue
        elif agent8.isPrey(agent8.loc, prey8.loc):
            print(True)
            ans += 1
            continue

    cnt += 1
    survival.append(count)
    print("cnt: ", i, " Steps : ", count)

print("============================")
print("CNT: ", cnt)
print("TRUE: ", ans)
print("FALSE: ", cnt-(ans+hung))
print("HUNG: ", hung)
avg_steps = sum(survival)/len(survival)
print("AVG. STEPS: ", avg_steps)
print("============================")


In [ ]:
# threshold = 100
node_num = 50

cnt = 0
hung = 0
ans = 0

survival = []

for i in range(loops):
    g = Graph(node_num)
    prey = Prey(g)
    pred = Predator(g)
    prey7 = copy.copy(prey)
    pred7 = copy.copy(pred)

    
    prey_belief = [1/(g.node_num-1)]*(g.node_num) ### [1/49]*50
    pred_belief = [1/(g.node_num-1)]*(g.node_num)

    ##------ AGENT-7 RUN MAIN ------##
    agent7 = Agent7(g, prey7.loc, pred7.loc, prey_belief, pred_belief, False)

    count = 0

    while agent7.loc != pred7.loc and agent7.loc != prey7.loc:
        if count > threshold:
            print("HUNG SIMULATION")
            # break
            hung += 1
            continue

        agent7.debug("BEFORE: ", agent7.loc, prey7.loc, pred7.loc)
        agent7.loc = agent7.move(prey7.loc, pred7.loc, pred_belief, prey_belief)

        if agent7.isPred(agent7.loc, pred7.loc) and agent7.isPrey(agent7.loc, prey7.loc):
            print(True)
            ans += 1
            continue
        elif agent7.isPred(agent7.loc, pred7.loc):
            print(False)
            continue
        elif agent7.isPrey(agent7.loc, prey7.loc):
            print(True)
            ans += 1
            continue

        prey7.loc = prey7.move()
        prey_belief = agent7.prey_move_prey_belief(prey7.loc, prey_belief)
        pred7.loc = pred7.move(agent7.loc)
        pred_belief = agent7.pred_move_pred_belief(pred7.loc, pred_belief)
        agent7.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
        
        agent7.debug("After: ", agent7.loc, prey7.loc, pred7.loc) ##
        agent7.debug("prey7 found : ", agent7.isPrey(agent7.loc, prey7.loc)," ; ", "pred5 found : ", agent7.isPred(agent7.loc, pred7.loc)) ##

        count = count + 1

        if agent7.isPred(agent7.loc, pred7.loc) and agent7.isPrey(agent7.loc, prey7.loc):
            print(True)
            ans += 1
            continue
        elif agent7.isPred(agent7.loc, pred7.loc):
            print(False)
            continue
        elif agent7.isPrey(agent7.loc, prey7.loc):
            print(True)
            ans += 1
            continue

    cnt += 1
    survival.append(count)
    print("cnt: ", i, " Steps : ", count)

print("============================")
print("CNT: ", cnt)
print("TRUE: ", ans)
print("FALSE: ", cnt-(ans+hung))
print("HUNG: ", hung)
avg_steps = sum(survival)/len(survival)
print("AVG. STEPS: ", avg_steps)
print("============================")


In [ ]:
# threshold = 100
node_num = 50

cnt = 0
hung = 0
ans = 0

survival = []

for i in range(loops):
    g = Graph(node_num)
    prey = Prey(g)
    pred = Predator(g)
    prey6 = copy.copy(prey)
    pred6 = copy.copy(pred)

    
    # prey_belief = [1/(g.node_num-1)]*(g.node_num) ### [1/49]*50
    pred_belief = [1/(g.node_num-1)]*(g.node_num)    
    
    ##------ AGENT-6 RUN MAIN ------##
    agent6 = Agent6(g, prey6.loc, pred6.loc, pred_belief, False)

    count = 0

    while agent6.loc != pred6.loc and agent6.loc != prey6.loc:
        if count > threshold:
            print("HUNG SIMULATION")
            hung += 1
            continue

        agent6.debug("BEFORE: ", agent6.loc, prey6.loc, pred6.loc)
        
        agent6.loc = agent6.move(prey6.loc, pred6.loc, pred_belief)

        if agent6.isPred(agent6.loc, pred6.loc) and agent6.isPrey(agent6.loc, prey6.loc):
            print(True)
            ans += 1
            continue
        elif agent6.isPred(agent6.loc, pred6.loc):
            print(False)
            continue
        elif agent6.isPrey(agent6.loc, prey6.loc):
            print(True)
            ans += 1
            continue    
        
        prey6.loc = prey6.move()
        pred6.loc = pred6.move(agent6.loc)
        pred_belief = agent6.pred_move_pred_belief(pred6.loc, pred_belief)
        agent6.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
        
        agent6.debug("After: ", agent6.loc, prey6.loc, pred6.loc) ##
        agent6.debug("prey6 found : ", agent6.isPrey(agent6.loc, prey6.loc)," ; ", "pred6 found : ", agent6.isPred(agent6.loc, pred6.loc)) ##

        count = count + 1

        if agent6.isPred(agent6.loc, pred6.loc) and agent6.isPrey(agent6.loc, prey6.loc):
            print(True)
            ans += 1
            continue
        elif agent6.isPred(agent6.loc, pred6.loc):
            print(False)
            continue
        elif agent6.isPrey(agent6.loc, prey6.loc):
            print(True)
            ans += 1
            continue
        
    cnt += 1
    survival.append(count)

    print("cnt: ", cnt, "Steps : ", count)


print("============================")
print("CNT: ", cnt)
print("TRUE: ", ans)
print("FALSE: ", cnt-(ans+hung))
print("HUNG: ", hung)
avg_steps = sum(survival)/len(survival)
print("AVG. STEPS: ", avg_steps)
print("============================")


Streaming output truncated to the last 5000 lines.
cnt:  504 Steps :  12
False
cnt:  505 Steps :  21
False
cnt:  506 Steps :  11
True
cnt:  507 Steps :  7
True
cnt:  508 Steps :  4
False
cnt:  509 Steps :  7
False
cnt:  510 Steps :  4
False
cnt:  511 Steps :  13
False
cnt:  512 Steps :  3
False
cnt:  513 Steps :  11
False
cnt:  514 Steps :  0
False
cnt:  515 Steps :  11
True
cnt:  516 Steps :  7
True
cnt:  517 Steps :  0
False
cnt:  518 Steps :  14
True
cnt:  519 Steps :  1
True
cnt:  520 Steps :  9
True
cnt:  521 Steps :  10
False
cnt:  522 Steps :  4
True
cnt:  523 Steps :  2
True
cnt:  524 Steps :  13
False
cnt:  525 Steps :  12
False
cnt:  526 Steps :  16
False
cnt:  527 Steps :  9
False
cnt:  528 Steps :  5
False
cnt:  529 Steps :  9
True
cnt:  530 Steps :  9
False
cnt:  531 Steps :  8
False
cnt:  532 Steps :  5
True
cnt:  533 Steps :  0
False
cnt:  534 Steps :  2
False
cnt:  535 Steps :  8
True
cnt:  536 Steps :  2
True
cnt:  537 Steps :  15
False
cnt:  538 Steps :  15
True
cnt: 

In [ ]:
# threshold = 100
node_num = 50

cnt = 0
hung = 0
ans = 0

survival = []

for i in range(loops):
    g = Graph(node_num)
    prey = Prey(g)
    pred = Predator(g)
    prey5 = copy.copy(prey)
    pred5 = copy.copy(pred)

    
    # prey_belief = [1/(g.node_num-1)]*(g.node_num) ### [1/49]*50
    pred_belief = [1/(g.node_num-1)]*(g.node_num)

    ##------ AGENT-5 RUN MAIN ------##
    agent5 = Agent5(g, prey5.loc, pred5.loc, pred_belief, False)

    count = 0

    while agent5.loc != pred5.loc and agent5.loc != prey5.loc:
        if count > threshold:
            print("HUNG SIMULATION")
            hung += 1
            continue

        agent5.debug("BEFORE: ", agent5.loc, prey5.loc, pred5.loc)
        
        agent5.loc = agent5.move(prey5.loc, pred5.loc, pred_belief)

        if agent5.isPred(agent5.loc, pred5.loc) and agent5.isPrey(agent5.loc, prey5.loc):
            print(True)
            ans += 1
            continue
        elif agent5.isPred(agent5.loc, pred5.loc):
            print(False)
            continue
        elif agent5.isPrey(agent5.loc, prey5.loc):
            print(True)
            ans += 1
            continue

        prey5.loc = prey5.move()
        pred5.loc = pred5.move(agent5.loc)
        pred_belief = agent5.pred_move_pred_belief(pred5.loc, pred_belief)
        agent5.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
        
        agent5.debug("After: ", agent5.loc, prey5.loc, pred5.loc) ##
        agent5.debug("prey5 found : ", agent5.isPrey(agent5.loc, prey5.loc)," ; ", "pred5 found : ", agent5.isPred(agent5.loc, pred5.loc)) ##

        count = count + 1

        if agent5.isPred(agent5.loc, pred5.loc) and agent5.isPrey(agent5.loc, prey5.loc):
            print(True)
            ans += 1
            continue
        elif agent5.isPred(agent5.loc, pred5.loc):
            print(False)
            continue
        elif agent5.isPrey(agent5.loc, prey5.loc):
            print(True)
            ans += 1
            continue

    cnt += 1
    survival.append(count)
    print("cnt: ", cnt, "Steps : ", count)


print("============================")
print("CNT: ", cnt)
print("TRUE: ", ans)
print("FALSE: ", cnt-(ans+hung))
print("HUNG: ", hung)
avg_steps = sum(survival)/len(survival)
print("AVG. STEPS: ", avg_steps)
print("============================")


Streaming output truncated to the last 5000 lines.
cnt:  504 Steps :  4
True
cnt:  505 Steps :  17
True
cnt:  506 Steps :  8
False
cnt:  507 Steps :  1
False
cnt:  508 Steps :  9
True
cnt:  509 Steps :  0
False
cnt:  510 Steps :  1
True
cnt:  511 Steps :  2
True
cnt:  512 Steps :  0
True
cnt:  513 Steps :  8
True
cnt:  514 Steps :  3
True
cnt:  515 Steps :  15
False
cnt:  516 Steps :  12
True
cnt:  517 Steps :  2
False
cnt:  518 Steps :  0
True
cnt:  519 Steps :  14
True
cnt:  520 Steps :  15
False
cnt:  521 Steps :  2
True
cnt:  522 Steps :  3
False
cnt:  523 Steps :  4
True
cnt:  524 Steps :  2
True
cnt:  525 Steps :  2
False
cnt:  526 Steps :  14
False
cnt:  527 Steps :  15
True
cnt:  528 Steps :  18
False
cnt:  529 Steps :  6
True
cnt:  530 Steps :  15
True
cnt:  531 Steps :  0
True
cnt:  532 Steps :  13
False
cnt:  533 Steps :  6
False
cnt:  534 Steps :  12
False
cnt:  535 Steps :  13
False
cnt:  536 Steps :  11
False
cnt:  537 Steps :  17
True
cnt:  538 Steps :  0
True
cnt:  539 

In [ ]:
# threshold = 100
node_num = 50

cnt = 0
hung = 0
ans = 0

survival = []

for i in range(loops):
    g = Graph(node_num)
    prey = Prey(g)
    pred = Predator(g)
    prey4 = copy.copy(prey)
    pred4 = copy.copy(pred)

    
    prey_belief = [1/(g.node_num-1)]*(g.node_num) ### [1/49]*50
    # pred_belief = [1/(g.node_num-1)]*(g.node_num)    
    
    ##------ AGENT-4 RUN MAIN ------##
    agent4 = Agent4(g, prey4.loc, pred4.loc, prey_belief, False)

    tmp_prey_belief = prey_belief

    count = 0

    while agent4.loc != pred4.loc and agent4.loc != prey4.loc:
        if count > threshold:
            print("HUNG SIMULATION")
            hung += 1
            continue

        agent4.debug("Before: ", agent4.loc, prey4.loc, pred4.loc) ##
        
        agent4.loc = agent4.move(prey4.loc, pred4.loc, prey_belief, tmp_prey_belief)
        
        if agent4.isPred(agent4.loc, pred4.loc) and agent4.isPrey(agent4.loc, prey4.loc):
            print(True)
            ans += 1
            continue
        elif agent4.isPred(agent4.loc, pred4.loc):
            print(False)
            continue
        elif agent4.isPrey(agent4.loc, prey4.loc):
            print(True)
            ans += 1
            continue
        
        prey4.loc = prey4.move()
        prey_belief = agent4.prey_move_prey_belief(prey4.loc, prey_belief)
        agent4.debug('Prey Belief after P: ', prey_belief, " ; SUM :", sum(prey_belief))
        pred4.loc = pred4.move(agent4.loc)
        
        agent4.debug("After: ", agent4.loc, prey4.loc, pred4.loc) ##
        agent4.debug("prey4 found : ", agent4.isPrey(agent4.loc, prey4.loc)," ; ", "pred4 found : ", agent4.isPred(agent4.loc, pred4.loc)) ##
        
        tmp_prey_belief = agent4.prey_move_prey_belief(prey4.loc, prey_belief)
        agent4.debug('TMP_BELIEF: ', tmp_prey_belief, 'SUM: ', sum(tmp_prey_belief))
        
        count = count + 1

        if agent4.isPred(agent4.loc, pred4.loc) and agent4.isPrey(agent4.loc, prey4.loc):
            print(True)
            ans += 1
            continue
        elif agent4.isPred(agent4.loc, pred4.loc):
            print(False)
            continue
        elif agent4.isPrey(agent4.loc, prey4.loc):
            print(True)
            ans += 1
            continue

    cnt += 1
    survival.append(count)
    print("cnt: ", cnt, "Steps : ", count)


print("============================")
print("CNT: ", cnt)
print("TRUE: ", ans)
print("FALSE: ", cnt-(ans+hung))
print("HUNG: ", hung)
avg_steps = sum(survival)/len(survival)
print("AVG. STEPS: ", avg_steps)
print("============================")


In [ ]:
threshold = 5000
loops = 3000
node_num = 50

cnt = 0
hung = 0
ans = 0

survival = []

for i in range(loops):
    g = Graph(node_num)
    prey = Prey(g)
    pred = Predator(g)
    prey3 = copy.copy(prey)
    pred3 = copy.copy(pred)

    
    prey_belief = [1/(g.node_num-1)]*(g.node_num) ### [1/49]*50
    # pred_belief = [1/(g.node_num-1)]*(g.node_num)    

    ##------ AGENT-3 RUN MAIN ------##
    agent3 = Agent3(g, prey3.loc, pred3.loc, prey_belief, False)

    count = 0
    while agent3.loc != pred3.loc and agent3.loc != prey3.loc:
        if count > threshold:
            print("HUNG SIMULATION")
            hung += 1
            continue

        agent3.debug("Before: ", agent3.loc, prey3.loc, pred3.loc) ##
        
        agent3.loc = agent3.move(prey3.loc, pred3.loc, prey_belief)
        
        if agent3.isPred(agent3.loc, pred3.loc) and agent3.isPrey(agent3.loc, prey3.loc):
            print(True)
            ans += 1
            continue
        elif agent3.isPred(agent3.loc, pred3.loc):
            print(False)
            continue
        elif agent3.isPrey(agent3.loc, prey3.loc):
            print(True)
            ans += 1
            continue

        prey3.loc = prey3.move()
        prey_belief = agent3.prey_move_prey_belief(prey3.loc, prey_belief)
        agent3.debug('Prey Belief after P: ', prey_belief, " ; SUM :", sum(prey_belief))
        pred3.loc = pred3.move(agent3.loc)
        
        agent3.debug("After: ", agent3.loc, prey3.loc, pred3.loc) ##
        agent3.debug("prey3 found : ", agent3.isPrey(agent3.loc, prey3.loc)," ; ", "pred3 found : ", agent3.isPred(agent3.loc, pred3.loc)) ##
        
        count = count + 1

        if agent3.isPred(agent3.loc, pred3.loc) and agent3.isPrey(agent3.loc, prey3.loc):
            print(True)
            ans += 1
            continue
        elif agent3.isPred(agent3.loc, pred3.loc):
            print(False)
            continue
        elif agent3.isPrey(agent3.loc, prey3.loc):
            print(True)
            ans += 1
            continue


    cnt += 1
    survival.append(count)
    print("cnt: ", cnt, "Steps : ", count)


print("============================")
print("CNT: ", cnt)
print("TRUE: ", ans)
print("FALSE: ", cnt-(ans+hung))
print("HUNG: ", hung)
avg_steps = sum(survival)/len(survival)
print("AVG. STEPS: ", avg_steps)
print("============================")

In [ ]:
threshold = 5000
loops = 3000
node_num = 50

cnt = 0
hung = 0
ans = 0

survival = []

for i in range(loops):
    g = Graph(node_num)
    prey = Prey(g)
    pred = Predator(g)
    prey2 = copy.copy(prey)
    pred2 = copy.copy(pred)

    ##------ AGENT-2 RUN ------##

    agent2 = Agent2(g, prey2.loc, pred2.loc, False)

    count = 0

    while agent2.loc != pred2.loc and agent2.loc != prey2.loc:
        if count > threshold:
            print("HUNG SIMULATION")
            hung += 1
            continue

        agent2.debug("Before: ", agent2.loc, prey2.loc, pred2.loc)
        agent2.loc = agent2.move(prey2.loc, pred2.loc)

        if agent2.isPred(pred2.loc) and agent2.isPrey(prey2.loc):
            print(True)
            ans += 1
            continue
        elif agent2.isPred(pred2.loc):
            print(False)
            continue
        elif agent2.isPrey(prey2.loc):
            print(True)
            ans += 1
            continue        

        prey2.loc = prey2.move()
        pred2.loc = pred2.move(agent2.loc)
        agent2.debug("After: ", agent2.loc, prey2.loc, pred2.loc)
        # print("After: ", agent2.move(prey2.loc, pred2.loc), prey2.move(), pred2.move(agent2.loc))
        agent2.debug("prey2 found : ", agent2.isPrey(prey2.loc)," ; ", "pred2 found : ", agent2.isPred(pred2.loc))
        
        count = count + 1
        
        if agent2.isPred(pred2.loc) and agent2.isPrey(prey2.loc):
            print(True)
            ans += 1
            continue
        elif agent2.isPred(pred2.loc):
            print(False)
            continue
        elif agent2.isPrey(prey2.loc):
            print(True)
            ans += 1
            continue        

    cnt += 1
    survival.append(count)

    print("cnt: ", cnt, "Steps : ", count)


print("============================")
print("CNT: ", cnt)
print("TRUE: ", ans)
print("FALSE: ", cnt-(ans+hung))
print("HUNG: ", hung)
avg_steps = sum(survival)/len(survival)
print("AVG. STEPS: ", avg_steps)
print("============================")


Streaming output truncated to the last 5000 lines.
cnt:  504 Steps :  8
True
cnt:  505 Steps :  60
True
cnt:  506 Steps :  37
True
cnt:  507 Steps :  308
True
cnt:  508 Steps :  6
True
cnt:  509 Steps :  28
True
cnt:  510 Steps :  5
True
cnt:  511 Steps :  58
True
cnt:  512 Steps :  24
True
cnt:  513 Steps :  715
True
cnt:  514 Steps :  12
True
cnt:  515 Steps :  42
True
cnt:  516 Steps :  14
True
cnt:  517 Steps :  33
False
cnt:  518 Steps :  70
True
cnt:  519 Steps :  24
True
cnt:  520 Steps :  166
True
cnt:  521 Steps :  20
True
cnt:  522 Steps :  134
True
cnt:  523 Steps :  91
True
cnt:  524 Steps :  266
True
cnt:  525 Steps :  6
True
cnt:  526 Steps :  243
True
cnt:  527 Steps :  111
True
cnt:  528 Steps :  8
True
cnt:  529 Steps :  17
True
cnt:  530 Steps :  239
True
cnt:  531 Steps :  9
True
cnt:  532 Steps :  27
True
cnt:  533 Steps :  20
True
cnt:  534 Steps :  26
True
cnt:  535 Steps :  2
True
cnt:  536 Steps :  251
True
cnt:  537 Steps :  1
True
cnt:  538 Steps :  32
True
cn

In [ ]:
threshold = 5000
loops = 3000
node_num = 50

cnt = 0
hung = 0
ans = 0

survival = []

for i in range(loops):
    g = Graph(node_num)
    prey = Prey(g)
    pred = Predator(g)
    prey1 = copy.copy(prey)
    pred1 = copy.copy(pred)

    ##------ AGENT-1 RUN ------##

    agent1 = Agent1(g, prey1.loc, pred1.loc, False)

    count = 0

    while agent1.loc != pred1.loc and agent1.loc != prey1.loc:
        if count > threshold:
            print("HUNG SIMULATION")
            hung += 1
            continue

        agent1.debug("Before: ", agent1.loc, prey1.loc, pred1.loc)
        agent1.loc = agent1.move(prey1.loc, pred1.loc)

        if agent1.isPred(pred1.loc) and agent1.isPrey(prey1.loc):
            print(True)
            ans += 1
            continue
        elif agent1.isPred(pred1.loc):
            print(False)
            continue
        elif agent1.isPrey(prey1.loc):
            print(True)
            ans += 1
            continue        

        prey1.loc = prey1.move()
        pred1.loc = pred1.move(agent1.loc)
        agent1.debug("After: ", agent1.loc, prey1.loc, pred1.loc)
        # print("After: ", agent1.move(prey1.loc, pred1.loc), prey1.move(), pred1.move(agent1.loc))
        agent1.debug("prey1 found : ", agent1.isPrey(prey1.loc)," ; ", "pred1 found : ", agent1.isPred(pred1.loc))
        
        count = count + 1
        
        if agent1.isPred(pred1.loc) and agent1.isPrey(prey1.loc):
            print(True)
            ans += 1
            continue
        elif agent1.isPred(pred1.loc):
            print(False)
            continue
        elif agent1.isPrey(prey1.loc):
            print(True)
            ans += 1
            continue        

    cnt += 1
    survival.append(count)
    print("cnt: ", cnt, "Steps : ", count)


print("============================")
print("CNT: ", cnt)
print("TRUE: ", ans)
print("FALSE: ", cnt-(ans+hung))
print("HUNG: ", hung)
avg_steps = sum(survival)/len(survival)
print("AVG. STEPS: ", avg_steps)
print("============================")


Streaming output truncated to the last 5000 lines.
cnt:  504 Steps :  12
True
cnt:  505 Steps :  11
True
cnt:  506 Steps :  5
True
cnt:  507 Steps :  4
True
cnt:  508 Steps :  11
True
cnt:  509 Steps :  9
True
cnt:  510 Steps :  12
True
cnt:  511 Steps :  5
False
cnt:  512 Steps :  4
True
cnt:  513 Steps :  11
True
cnt:  514 Steps :  6
True
cnt:  515 Steps :  1
True
cnt:  516 Steps :  19
True
cnt:  517 Steps :  9
True
cnt:  518 Steps :  2
True
cnt:  519 Steps :  21
True
cnt:  520 Steps :  11
True
cnt:  521 Steps :  17
True
cnt:  522 Steps :  9
True
cnt:  523 Steps :  8
True
cnt:  524 Steps :  8
True
cnt:  525 Steps :  23
True
cnt:  526 Steps :  9
True
cnt:  527 Steps :  7
False
cnt:  528 Steps :  14
True
cnt:  529 Steps :  22
True
cnt:  530 Steps :  18
True
cnt:  531 Steps :  20
True
cnt:  532 Steps :  8
True
cnt:  533 Steps :  1
False
cnt:  534 Steps :  4
True
cnt:  535 Steps :  3
True
cnt:  536 Steps :  30
True
cnt:  537 Steps :  3
True
cnt:  538 Steps :  13
True
cnt:  539 Steps :  1

In [ ]:
#@title
##------ AGENT-1 RUN ------##

agent1 = Agent1(g, prey1.loc, pred1.loc, True)

count = 0
while agent1.loc != pred1.loc and agent1.loc != prey1.loc:
    if count > threshold:
        print("HUNG SIMULATION")
        break
        
    print("Before: ", agent1.loc, prey1.loc, pred1.loc)
    agent1.loc = agent1.move(prey1.loc, pred1.loc)
    prey1.loc = prey1.move()
    pred1.loc = pred1.move(agent1.loc)
    print("After: ", agent1.loc, prey1.loc, pred1.loc)
    # print("After: ", agent1.move(prey1.loc, pred1.loc), prey1.move(), pred1.move(agent1.loc))
    print("prey1 found : ", agent1.isPrey(prey1.loc)," ; ", "pred1 found : ", agent1.isPred(pred1.loc))
    
    count = count + 1
    
    if agent1.isPred(pred1.loc) or agent1.isPrey(prey1.loc):
        break

print("Steps : ", count)

**REFERENCES:**

[Graph Implementation](https://stackabuse.com/courses/graphs-in-python-theory-and-implementation/lessons/representing-graphs-in-code/)

In [ ]:
#@title


In [ ]:
#@title


In [ ]:
#@title
# ##------ AGENT-7 RUN MAIN ------##
# agent7 = Agent7(g, prey7.loc, pred7.loc, prey_belief, pred_belief, True)

# count = 0

# while agent7.loc != pred7.loc and agent7.loc != prey7.loc:
#     if count > threshold:
#         print("HUNG SIMULATION")
#         break

#     agent7.debug("BEFORE: ", agent7.loc, prey7.loc, pred7.loc)
#     agent7.loc = agent7.move(prey7.loc, pred7.loc, pred_belief, prey_belief)
    
#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

#     prey7.loc = prey7.move()
#     prey_belief = agent7.prey_move_prey_belief(prey7.loc, prey_belief)
#     pred7.loc = pred7.move(agent7.loc)
#     pred_belief = agent7.pred_move_pred_belief(pred7.loc, pred_belief)
#     agent7.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
    
#     agent7.debug("After: ", agent7.loc, prey7.loc, pred7.loc) ##
#     agent7.debug("prey7 found : ", agent7.isPrey(agent7.loc, prey7.loc)," ; ", "pred5 found : ", agent7.isPred(agent7.loc, pred7.loc)) ##

#     count = count + 1

#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

# print("Steps : ", count)


In [ ]:
#@title
# ##------ AGENT-7 RUN MAIN ------##
# agent7 = Agent7(g, prey7.loc, pred7.loc, prey_belief, pred_belief, True)

# count = 0

# while agent7.loc != pred7.loc and agent7.loc != prey7.loc:
#     if count > threshold:
#         print("HUNG SIMULATION")
#         break

#     agent7.debug("BEFORE: ", agent7.loc, prey7.loc, pred7.loc)
#     agent7.loc = agent7.move(prey7.loc, pred7.loc, pred_belief, prey_belief)
    
#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

#     prey7.loc = prey7.move()
#     prey_belief = agent7.prey_move_prey_belief(prey7.loc, prey_belief)
#     pred7.loc = pred7.move(agent7.loc)
#     pred_belief = agent7.pred_move_pred_belief(pred7.loc, pred_belief)
#     agent7.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
    
#     agent7.debug("After: ", agent7.loc, prey7.loc, pred7.loc) ##
#     agent7.debug("prey7 found : ", agent7.isPrey(agent7.loc, prey7.loc)," ; ", "pred5 found : ", agent7.isPred(agent7.loc, pred7.loc)) ##

#     count = count + 1

#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

# print("Steps : ", count)


In [ ]:
#@title
# ##------ AGENT-7 RUN MAIN ------##
# agent7 = Agent7(g, prey7.loc, pred7.loc, prey_belief, pred_belief, True)

# count = 0

# while agent7.loc != pred7.loc and agent7.loc != prey7.loc:
#     if count > threshold:
#         print("HUNG SIMULATION")
#         break

#     agent7.debug("BEFORE: ", agent7.loc, prey7.loc, pred7.loc)
#     agent7.loc = agent7.move(prey7.loc, pred7.loc, pred_belief, prey_belief)
    
#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

#     prey7.loc = prey7.move()
#     prey_belief = agent7.prey_move_prey_belief(prey7.loc, prey_belief)
#     pred7.loc = pred7.move(agent7.loc)
#     pred_belief = agent7.pred_move_pred_belief(pred7.loc, pred_belief)
#     agent7.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
    
#     agent7.debug("After: ", agent7.loc, prey7.loc, pred7.loc) ##
#     agent7.debug("prey7 found : ", agent7.isPrey(agent7.loc, prey7.loc)," ; ", "pred5 found : ", agent7.isPred(agent7.loc, pred7.loc)) ##

#     count = count + 1

#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

# print("Steps : ", count)


In [ ]:
#@title
# ##------ AGENT-7 RUN MAIN ------##
# agent7 = Agent7(g, prey7.loc, pred7.loc, prey_belief, pred_belief, True)

# count = 0

# while agent7.loc != pred7.loc and agent7.loc != prey7.loc:
#     if count > threshold:
#         print("HUNG SIMULATION")
#         break

#     agent7.debug("BEFORE: ", agent7.loc, prey7.loc, pred7.loc)
#     agent7.loc = agent7.move(prey7.loc, pred7.loc, pred_belief, prey_belief)
    
#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

#     prey7.loc = prey7.move()
#     prey_belief = agent7.prey_move_prey_belief(prey7.loc, prey_belief)
#     pred7.loc = pred7.move(agent7.loc)
#     pred_belief = agent7.pred_move_pred_belief(pred7.loc, pred_belief)
#     agent7.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
    
#     agent7.debug("After: ", agent7.loc, prey7.loc, pred7.loc) ##
#     agent7.debug("prey7 found : ", agent7.isPrey(agent7.loc, prey7.loc)," ; ", "pred5 found : ", agent7.isPred(agent7.loc, pred7.loc)) ##

#     count = count + 1

#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

# print("Steps : ", count)


In [ ]:
#@title
# ##------ AGENT-7 RUN MAIN ------##
# agent7 = Agent7(g, prey7.loc, pred7.loc, prey_belief, pred_belief, True)

# count = 0

# while agent7.loc != pred7.loc and agent7.loc != prey7.loc:
#     if count > threshold:
#         print("HUNG SIMULATION")
#         break

#     agent7.debug("BEFORE: ", agent7.loc, prey7.loc, pred7.loc)
#     agent7.loc = agent7.move(prey7.loc, pred7.loc, pred_belief, prey_belief)
#     prey7.loc = prey7.move()

#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break
    
#     prey_belief = agent7.prey_move_prey_belief(prey7.loc, prey_belief)
#     pred7.loc = pred7.move(agent7.loc)
#     pred_belief = agent7.pred_move_pred_belief(pred7.loc, pred_belief)
#     agent7.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
    
#     agent7.debug("After: ", agent7.loc, prey7.loc, pred7.loc) ##
#     agent7.debug("prey7 found : ", agent7.isPrey(agent7.loc, prey7.loc)," ; ", "pred5 found : ", agent7.isPred(agent7.loc, pred7.loc)) ##

#     count = count + 1

#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

# print("Steps : ", count)


In [ ]:
#@title
# ##------ AGENT-7 RUN MAIN ------##
# agent7 = Agent7(g, prey7.loc, pred7.loc, prey_belief, pred_belief, True)

# count = 0

# while agent7.loc != pred7.loc and agent7.loc != prey7.loc:
#     if count > threshold:
#         print("HUNG SIMULATION")
#         break

#     agent7.debug("BEFORE: ", agent7.loc, prey7.loc, pred7.loc)
#     agent7.loc = agent7.move(prey7.loc, pred7.loc, pred_belief, prey_belief)
#     prey7.loc = prey7.move()
#     prey_belief = agent7.prey_move_prey_belief(prey7.loc, prey_belief)
#     pred7.loc = pred7.move(agent7.loc)
#     pred_belief = agent7.pred_move_pred_belief(pred7.loc, pred_belief)
#     agent7.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
    
#     agent7.debug("After: ", agent7.loc, prey7.loc, pred7.loc) ##
#     agent7.debug("prey7 found : ", agent7.isPrey(agent7.loc, prey7.loc)," ; ", "pred5 found : ", agent7.isPred(agent7.loc, pred7.loc)) ##

#     count = count + 1

#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break
        
# print("Steps : ", count)


In [ ]:
#@title
# ##------ AGENT-7 RUN MAIN ------##
# agent7 = Agent7(g, prey7.loc, pred7.loc, prey_belief, pred_belief, True)

# count = 0

# while agent7.loc != pred7.loc and agent7.loc != prey7.loc:
#     if count > threshold:
#         print("HUNG SIMULATION")
#         break

#     agent7.debug("BEFORE: ", agent7.loc, prey7.loc, pred7.loc)
#     agent7.loc = agent7.move(prey7.loc, pred7.loc, pred_belief, prey_belief)
#     prey7.loc = prey7.move()
#     prey_belief = agent7.prey_move_prey_belief(prey7.loc, prey_belief)
#     pred7.loc = pred7.move(agent7.loc)
#     pred_belief = agent7.pred_move_pred_belief(pred7.loc, pred_belief)
#     agent7.debug('Pred Belief after P: ', pred_belief, " ; SUM :", sum(pred_belief))
    
#     agent7.debug("After: ", agent7.loc, prey7.loc, pred7.loc) ##
#     agent7.debug("prey7 found : ", agent7.isPrey(agent7.loc, prey7.loc)," ; ", "pred5 found : ", agent7.isPred(agent7.loc, pred7.loc)) ##

#     count = count + 1

#     if agent7.isPred(agent7.loc, pred7.loc) or agent7.isPrey(agent7.loc, prey7.loc):
#         break

# print("Steps : ", count)
